In [2]:
import pandas as pd
import numpy as np

In [26]:
# 유저테이블 불러오기
click = pd.read_csv("/Users/t2023-m0052/Documents/GitHub/final_project/data/유저테이블.csv")

# media portfolio 불러오기
media_portfolio = pd.read_csv("/Users/t2023-m0052/Documents/GitHub/final_project/data/media_performance_classification.csv")
media_portfolio = media_portfolio.iloc[:, 2:]

In [121]:
ads_list = pd.read_csv('/Users/t2023-m0052/Documents/GitHub/final_project/data/광고도메인리스트.csv')
ads_list = ads_list.iloc[:, 1:]

In [4]:
def analyze_ads_performance(ads_idx, click_data, media_portfolio=None):
    """
    특정 광고의 매체별 성과를 분석하는 함수
    """
    
    # 1. 해당 광고의 데이터가 있는지 확인
    ads_data = click_data[click_data['ads_idx'] == ads_idx]
    if len(ads_data) == 0:
        print(f"광고 {ads_idx}에 대한 데이터가 없습니다.")
        return pd.DataFrame()
    
    # 2. 기본 성과 데이터 추출
    ads_performance = ads_data.groupby(['ads_idx', 'mda_idx']).agg({
        'click_key': 'count',
        'conversion': 'sum',
        'contract_price': 'first',
        'media_price': 'first',
        'domain': 'first',
        'ads_category': 'first'
    }).reset_index()
    
    # 컬럼명 변경
    ads_performance.columns = ['ads_idx', 'mda_idx', 'total_clicks', 'total_conversions', 
                              'contract_price', 'media_price', 'domain', 'ads_category']
    
    # 전환율 및 수익 계산
    ads_performance['cvr'] = (
        ads_performance['total_conversions'] / ads_performance['total_clicks']
    ).round(4)
    
    ads_performance['profit_per_conversion'] = (
        ads_performance['contract_price'] - ads_performance['media_price']
    )
    ads_performance['total_profit'] = (
        ads_performance['total_conversions'] * ads_performance['profit_per_conversion']
    )
    
    # 3. 날짜 범위 및 활동일 계산
    click_data_copy = click_data.copy()
    if not pd.api.types.is_datetime64_any_dtype(click_data_copy['click_date']):
        click_data_copy['click_date'] = pd.to_datetime(click_data_copy['click_date'])
    
    ads_activity = (
        click_data_copy.loc[click_data_copy['ads_idx'] == ads_idx]
                      .groupby('mda_idx')['click_date']
                      .agg(first_click='min', last_click='max')
                      .reset_index()
    )
    
    ads_activity['days_active_calc'] = (
        (ads_activity['last_click'] - ads_activity['first_click']).dt.days + 1
    )
    
    # 4. 데이터 병합
    merged = ads_performance.merge(
        ads_activity[['mda_idx', 'first_click', 'last_click', 'days_active_calc']],
        on='mda_idx', how='left'
    )
    
    # 5. 일평균 지표 계산
    merged['daily_clicks'] = merged['total_clicks'] / merged['days_active_calc']
    merged['daily_conversions'] = merged['total_conversions'] / merged['days_active_calc']
    merged['daily_profit'] = merged['total_profit'] / merged['days_active_calc']
    
    # 6. 배분 그룹 분류 (데이터가 충분한 경우에만)
    if len(merged) > 1:  # 최소 2개 이상의 매체가 있어야 중앙값 계산이 의미있음
        profit_median = merged['daily_profit'].median()
        conv_median = merged['daily_conversions'].median()
        
        merged['배분그룹'] = np.where(
            (merged['daily_profit'] >= profit_median) & (merged['daily_conversions'] >= conv_median),
            '잘 배분',
            '잘못 배분'
        )
        # 결과 정렬
        result = merged.sort_values(['배분그룹', 'daily_profit'], ascending=[True, False]).reset_index(drop=True)
    else:
        merged['배분그룹'] = '분류불가'
        result = merged.reset_index(drop=True)
    
    return result

In [5]:
ads_446909_analysis = analyze_ads_performance(446909, click)

In [147]:
ads_446909_analysis

,ads_idx,mda_idx,total_clicks,total_conversions,contract_price,media_price,domain,ads_category,cvr,profit_per_conversion,total_profit,first_click,last_click,days_active_calc,daily_clicks,daily_conversions,daily_profit,배분그룹
0,446909,563,58,7,18,14,커머스,4,0.1207,4,28,2025-08-25 10:45:25,2025-08-25 11:22:40,1,58.0,7.0,28.0,잘 배분
1,446909,562,4,0,18,14,커머스,4,0.0000,4,0,2025-08-25 10:48:50,2025-08-25 11:07:48,1,4.0,0.0,0.0,잘못 배분


In [7]:
ads_pool = pd.read_csv('/Users/t2023-m0052/Documents/GitHub/final_project/data/ads_pool.csv')
ads_pool = ads_pool.iloc[:, 2:]
ads_pool

,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,ads_order,ctit_mean,...,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster,mda_idx_arr,M,A
0,6508,1,1,2,0,10,금융,7,2381500,NaN,...,0,0,0,0,1,SMALL,0,NaN,0.0,1.0
1,6985,2,30,46,0,7,금융,7,1201938,NaN,...,0,0,3,0,5,MEDIUM,0,NaN,0.0,0.0
2,8327,1,6,11,0,10,미디어/컨텐츠,7,1673458,NaN,...,0,0,3,0,4,SMALL,0,NaN,0.0,1.0
3,9264,1,1,1,0,10,금융,7,2372800,NaN,...,0,0,0,0,1,SMALL,0,NaN,0.0,1.0
4,9716,2,4,6,1,8,생활,2,2699900,74.000000,...,0,1,1,0,4,SMALL,0,NaN,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4709,446899,2,52,63,49,4,커머스,7,1369916,42.102041,...,3,2,0,2,9,LARGE,0,NaN,0.0,0.0
4710,446900,2,44,59,41,4,식당/카페,7,1369915,48.512195,...,3,2,0,1,8,LARGE,0,NaN,0.0,0.0
4711,446902,1,2,2,0,8,미디어/컨텐츠,7,2735900,NaN,...,0,0,0,0,1,SMALL,0,NaN,0.0,0.0
4712,446908,1,1,1,0,0,금융,2,1682806,NaN,...,0,0,0,0,1,SMALL,0,NaN,0.0,0.0


In [215]:
ads_pool.columns

Index(['ads_idx', 'media_count', 'user_count', 'total_clicks',
       'total_conversions', 'ads_category', 'domain', 'ads_os_type',
       'ads_order', 'ctit_mean', 'ctit_median', 'ads_rejoin_type',
       'contract_price', 'media_price', 'first_click', 'last_click',
       'ads_name', 'ads_sdate', 'expire', 'days_active',
       'daily_avg_conversions', 'cvr', 'margin', 'roi', 'total_net_return',
       'ads_level_detailed', 'daily_clicks', 'daily_users', 'scale_level',
       'performance_level', 'ads_level', 'media_score', 'conv_score',
       'clicks_score', 'stability_score', 'cvr_score', 'total_score',
       'ads_size', 'cluster', 'mda_idx_arr', 'M', 'A'],
      dtype='object')

In [10]:
pd.set_option('display.max_columns', 50)

## 분석

In [11]:
ads_pool[(ads_pool['ads_size'] == 'LARGE') & (ads_pool['ads_category'] == 8)]

,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,ads_order,ctit_mean,ctit_median,ads_rejoin_type,contract_price,media_price,first_click,last_click,ads_name,ads_sdate,expire,days_active,daily_avg_conversions,cvr,margin,roi,total_net_return,ads_level_detailed,daily_clicks,daily_users,scale_level,performance_level,ads_level,media_score,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster,mda_idx_arr,M,A
5,9935,10,524,889,24,8,금융,7,2592600,145.083333,140.0,NONE,1500,1200,2025-07-26 00:39:57,2025-08-25 10:37:05,숨어있는 휴면포인트 조회,2021-02-01 10:00:00,0,31,0.774194,0.0,300,0.2,7200,MEGA,28.677419,16.903226,MEGA,LOW,MEGA_LOW,3,0,2,4,0,9,LARGE,0,NaN,0.0,1.0
26,13209,14,758,2162,169,8,금융,2,2496600,3529.295858,3527.0,NONE,1900,1425,2025-07-26 00:57:18,2025-08-25 10:06:34,아이부자,2021-10-01 17:00:00,0,31,5.451613,0.1,475,0.3,80275,MEGA,69.741935,24.451613,MEGA,LOW,MEGA_LOW,4,1,3,4,0,12,LARGE,0,22,1.0,1.0
31,14074,15,186,558,33,8,금융,2,2642800,1361.515152,1334.0,NONE,500,300,2025-07-26 00:49:28,2025-08-25 07:45:58,오토링 광고 1회 참여,2021-11-19 00:00:00,0,31,1.064516,0.1,200,0.7,6600,MEGA,18.000000,6.000000,LARGE,LOW,LARGE_LOW,4,1,2,4,0,11,LARGE,0,NaN,0.0,1.0
40,16523,6,659,1453,57,8,미디어/컨텐츠,7,2495400,4684.807018,238.0,NONE,1500,1000,2025-07-26 00:25:12,2025-08-25 06:24:46,탑툰,2022-04-14 14:00:00,0,31,1.838710,0.0,500,0.5,28500,LARGE,46.870968,21.258065,MEGA,LOW,MEGA_LOW,3,1,2,4,0,10,LARGE,0,NaN,0.0,1.0
42,16528,11,412,613,30,8,생활,7,1651118,889.666667,100.0,NONE,2000,1600,2025-07-26 02:01:55,2025-08-25 10:25:46,안심보호,2022-04-14 14:00:00,0,31,0.967742,0.0,400,0.2,12000,MEGA,19.774194,13.290323,LARGE,LOW,LARGE_LOW,3,0,2,4,0,9,LARGE,0,NaN,0.0,1.0
129,22232,7,614,1324,305,8,금융,2,2294400,338.950820,114.0,NONE,800,640,2025-07-26 00:46:23,2025-08-25 10:05:25,라이프플러스 트라이브,2023-06-05 14:00:00,0,31,9.838710,0.2,160,0.2,48800,LARGE,42.709677,19.806452,MEGA,LOW,MEGA_LOW,3,2,2,4,0,11,LARGE,0,NaN,0.0,1.0
133,22498,7,562,1050,65,8,미디어/컨텐츠,7,2718500,127.923077,97.0,NONE,4500,3200,2025-07-26 06:04:18,2025-08-25 11:11:39,미툰 7일 무료체험,2023-06-27 14:00:00,0,31,2.096774,0.1,1300,0.4,84500,LARGE,33.870968,18.129032,MEGA,LOW,MEGA_LOW,3,1,2,4,0,10,LARGE,0,NaN,0.0,1.0
173,28025,8,465,1566,3,8,게임,2,1668056,58.666667,62.0,NONE,3500,2625,2025-07-26 03:57:09,2025-08-25 11:16:49,게임 무료체험,2024-01-18 15:00:00,0,31,0.096774,0.0,875,0.3,2625,LARGE,50.516129,15.000000,MEGA,LOW,MEGA_LOW,3,0,2,4,0,9,LARGE,0,NaN,0.0,1.0
174,28026,9,95,138,1,8,미디어/컨텐츠,2,1672710,160.000000,160.0,NONE,800,600,2025-07-26 07:19:21,2025-08-25 09:21:34,판도라박스,2024-01-18 15:00:00,0,31,0.032258,0.0,200,0.3,200,LARGE,4.451613,3.064516,LARGE,LOW,LARGE_LOW,3,0,1,4,0,8,LARGE,0,NaN,0.0,1.0
190,29699,14,3490,11601,46,8,운세,7,2702100,183.869565,175.5,NONE,2800,2000,2025-07-26 00:03:09,2025-08-25 11:21:11,탑운세 7일 무료체험,2024-03-22 10:00:00,0,31,1.483871,0.0,800,0.4,36800,MEGA,374.225806,112.580645,MEGA,LOW,MEGA_LOW,4,1,3,4,0,12,LARGE,0,761,1.0,1.0


In [44]:
# 9935, 22498
ads_9935_analysis = analyze_ads_performance(9935, click)
ads_9935_analysis

,ads_idx,mda_idx,total_clicks,total_conversions,contract_price,media_price,domain,ads_category,cvr,profit_per_conversion,total_profit,first_click,last_click,days_active_calc,daily_clicks,daily_conversions,daily_profit,배분그룹
0,9935,667,775,20,1500,1200,금융,8,0.0258,300,6000,2025-07-26 00:39:57,2025-08-25 10:37:05,31,25.000000,0.645161,193.548387,잘 배분
1,9935,539,56,3,1500,1200,금융,8,0.0536,300,900,2025-08-11 19:02:46,2025-08-19 17:05:34,8,7.000000,0.375000,112.500000,잘 배분
2,9935,442,5,1,1500,1200,금융,8,0.2000,300,300,2025-07-30 13:28:34,2025-08-20 23:25:17,22,0.227273,0.045455,13.636364,잘 배분
3,9935,30,11,0,1500,1200,금융,8,0.0000,300,0,2025-07-29 00:31:09,2025-08-21 00:08:34,23,0.478261,0.000000,0.000000,잘 배분
4,9935,270,8,0,1500,840,금융,8,0.0000,660,0,2025-07-29 10:48:04,2025-08-23 16:16:55,26,0.307692,0.000000,0.000000,잘 배분
5,9935,281,15,0,1500,1200,금융,8,0.0000,300,0,2025-07-30 21:48:16,2025-08-19 23:08:11,21,0.714286,0.000000,0.000000,잘 배분
6,9935,482,1,0,1500,1200,금융,8,0.0000,300,0,2025-08-16 11:16:19,2025-08-16 11:16:19,1,1.000000,0.000000,0.000000,잘 배분
7,9935,645,2,0,1500,1200,금융,8,0.0000,300,0,2025-07-31 10:33:36,2025-07-31 15:19:13,1,2.000000,0.000000,0.000000,잘 배분
8,9935,761,15,0,1500,1200,금융,8,0.0000,300,0,2025-07-30 11:47:05,2025-08-24 20:52:12,26,0.576923,0.000000,0.000000,잘 배분
9,9935,854,1,0,1500,1200,금융,8,0.0000,300,0,2025-07-30 06:40:16,2025-07-30 06:40:16,1,1.000000,0.000000,0.000000,잘 배분


In [13]:
# 9935, 22498
ads_22498_analysis = analyze_ads_performance(22498, click)
ads_22498_analysis

,ads_idx,mda_idx,total_clicks,total_conversions,contract_price,media_price,domain,ads_category,cvr,profit_per_conversion,total_profit,first_click,last_click,days_active_calc,daily_clicks,daily_conversions,daily_profit,배분그룹
0,22498,667,997,61,4500,3200,미디어/컨텐츠,8,0.0612,1300,79300,2025-07-26 06:04:18,2025-08-25 11:11:39,31,32.161290,1.967742,2558.064516,잘 배분
1,22498,87,19,2,4500,3200,미디어/컨텐츠,8,0.1053,1300,2600,2025-07-27 11:01:31,2025-08-20 17:01:25,25,0.760000,0.080000,104.000000,잘 배분
2,22498,270,13,1,4500,2240,미디어/컨텐츠,8,0.0769,2260,2260,2025-07-30 05:50:41,2025-08-21 23:33:13,23,0.565217,0.043478,98.260870,잘 배분
3,22498,480,3,1,4500,3200,미디어/컨텐츠,8,0.3333,1300,1300,2025-08-10 23:45:15,2025-08-25 06:21:04,15,0.200000,0.066667,86.666667,잘 배분
4,22498,54,7,0,4500,3200,미디어/컨텐츠,8,0.0000,1300,0,2025-07-30 19:32:45,2025-08-24 17:11:58,25,0.280000,0.000000,0.000000,잘못 배분
5,22498,246,6,0,4500,3200,미디어/컨텐츠,8,0.0000,1300,0,2025-07-29 18:57:31,2025-08-15 21:56:34,18,0.333333,0.000000,0.000000,잘못 배분
6,22498,401,5,0,4500,3200,미디어/컨텐츠,8,0.0000,1300,0,2025-07-31 22:41:17,2025-08-12 23:02:01,13,0.384615,0.000000,0.000000,잘못 배분


In [45]:
ads_22498_analysis.sort_values(by = 'total_conversions', ascending= False)

,ads_idx,mda_idx,total_clicks,total_conversions,contract_price,media_price,domain,ads_category,cvr,profit_per_conversion,total_profit,first_click,last_click,days_active_calc,daily_clicks,daily_conversions,daily_profit,배분그룹
0,22498,667,997,61,4500,3200,미디어/컨텐츠,8,0.0612,1300,79300,2025-07-26 06:04:18,2025-08-25 11:11:39,31,32.161290,1.967742,2558.064516,잘 배분
1,22498,87,19,2,4500,3200,미디어/컨텐츠,8,0.1053,1300,2600,2025-07-27 11:01:31,2025-08-20 17:01:25,25,0.760000,0.080000,104.000000,잘 배분
2,22498,270,13,1,4500,2240,미디어/컨텐츠,8,0.0769,2260,2260,2025-07-30 05:50:41,2025-08-21 23:33:13,23,0.565217,0.043478,98.260870,잘 배분
3,22498,480,3,1,4500,3200,미디어/컨텐츠,8,0.3333,1300,1300,2025-08-10 23:45:15,2025-08-25 06:21:04,15,0.200000,0.066667,86.666667,잘 배분
4,22498,54,7,0,4500,3200,미디어/컨텐츠,8,0.0000,1300,0,2025-07-30 19:32:45,2025-08-24 17:11:58,25,0.280000,0.000000,0.000000,잘못 배분
5,22498,246,6,0,4500,3200,미디어/컨텐츠,8,0.0000,1300,0,2025-07-29 18:57:31,2025-08-15 21:56:34,18,0.333333,0.000000,0.000000,잘못 배분
6,22498,401,5,0,4500,3200,미디어/컨텐츠,8,0.0000,1300,0,2025-07-31 22:41:17,2025-08-12 23:02:01,13,0.384615,0.000000,0.000000,잘못 배분


In [42]:
ads_73878_analysis = analyze_ads_performance(73878, click)
ads_73878_analysis

,ads_idx,mda_idx,total_clicks,total_conversions,contract_price,media_price,domain,ads_category,cvr,profit_per_conversion,total_profit,first_click,last_click,days_active_calc,daily_clicks,daily_conversions,daily_profit,배분그룹
0,73878,14,1883,108,4000,3200,미디어/컨텐츠,8,0.0574,800,86400,2025-07-26 01:45:27,2025-08-25 10:34:32,31,60.741935,3.483871,2787.096774,잘 배분
1,73878,654,13309,40,4000,3200,미디어/컨텐츠,8,0.0030,800,32000,2025-07-26 00:03:38,2025-08-25 11:02:42,31,429.322581,1.290323,1032.258065,잘 배분
2,73878,56,418,20,4000,3200,미디어/컨텐츠,8,0.0478,800,16000,2025-07-26 00:04:02,2025-08-25 09:04:07,31,13.483871,0.645161,516.129032,잘 배분
3,73878,58,779,20,4000,3200,미디어/컨텐츠,8,0.0257,800,16000,2025-07-26 02:19:02,2025-08-25 11:21:00,31,25.129032,0.645161,516.129032,잘 배분
4,73878,667,370,19,4000,3200,미디어/컨텐츠,8,0.0514,800,15200,2025-07-26 17:19:57,2025-08-25 10:08:01,30,12.333333,0.633333,506.666667,잘 배분
5,73878,792,5,1,4000,3200,미디어/컨텐츠,8,0.2000,800,800,2025-07-27 21:28:54,2025-07-28 22:20:35,2,2.500000,0.500000,400.000000,잘 배분
6,73878,18,41,3,4000,3200,미디어/컨텐츠,8,0.0732,800,2400,2025-07-26 11:29:02,2025-08-21 05:03:38,26,1.576923,0.115385,92.307692,잘 배분
7,73878,761,110,3,4000,3200,미디어/컨텐츠,8,0.0273,800,2400,2025-07-26 08:55:35,2025-08-25 00:59:01,30,3.666667,0.100000,80.000000,잘 배분
8,73878,54,78,2,4000,3200,미디어/컨텐츠,8,0.0256,800,1600,2025-07-28 05:29:19,2025-08-24 23:21:50,28,2.785714,0.071429,57.142857,잘 배분
9,73878,270,37,1,4000,2500,미디어/컨텐츠,8,0.0270,1500,1500,2025-07-26 02:05:28,2025-08-24 23:30:51,30,1.233333,0.033333,50.000000,잘 배분


In [ ]:
# 667, 480, 87
ads_22498_analysis['mda_idx']

In [15]:
ads_pool[(ads_pool['ads_size'] == 'LARGE') & (ads_pool['ads_category'] == 5)]

,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,ads_order,ctit_mean,ctit_median,ads_rejoin_type,contract_price,media_price,first_click,last_click,ads_name,ads_sdate,expire,days_active,daily_avg_conversions,cvr,margin,roi,total_net_return,ads_level_detailed,daily_clicks,daily_users,scale_level,performance_level,ads_level,media_score,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster,mda_idx_arr,M,A
138,24944,6,302,1702,0,5,게임,2,2224400,NaN,NaN,NONE,1890,1400,2025-07-26 10:00:53,2025-08-25 10:34:03,걸 마스터,2023-09-11 16:00:00,0,31,0.000000,0.0,490,0.4,0,LARGE,54.903226,9.741935,MEGA,LOW,MEGA_LOW,3,0,2,4,0,9,LARGE,0,NaN,0.0,1.0
378,131233,7,150,1197,4,5,게임,2,2442000,1.151025e+04,9639.5,NONE,2040,1500,2025-07-26 00:24:18,2025-08-25 11:18:18,Supreme King: Earn Money,2024-12-23 17:00:00,0,31,0.129032,0.0,540,0.4,2160,LARGE,38.612903,4.838710,MEGA,LOW,MEGA_LOW,3,0,2,4,0,9,LARGE,0,NaN,0.0,1.0
396,140831,9,166,1258,5,5,게임,2,2446300,4.946146e+05,347532.0,NONE,18000,13000,2025-07-26 12:47:21,2025-08-25 10:27:20,RAID: Shadow Legends,2025-01-17 10:00:00,0,30,0.166667,0.0,5000,0.4,25000,LARGE,41.933333,5.533333,MEGA,LOW,MEGA_LOW,3,0,2,3,0,8,LARGE,0,"667,563,87,58,18,14,12",1.0,1.0
397,140832,11,1611,3897,83,5,게임,2,2449900,5.488301e+04,5606.0,NONE,7800,5800,2025-07-26 00:04:16,2025-08-25 10:57:00,MONEY CASH - 게임 플레이 및 돈 벌기,2025-01-17 10:00:00,0,31,2.677419,0.0,2000,0.3,166000,MEGA,125.709677,51.967742,MEGA,LOW,MEGA_LOW,3,1,3,4,0,11,LARGE,0,14,1.0,1.0
669,264408,12,192,268,77,5,게임,2,2451000,7.403584e+03,2791.0,NONE,2040,1500,2025-07-26 01:30:51,2025-08-25 09:09:34,마비노기 모바일,2025-04-03 16:00:00,0,31,2.483871,0.3,540,0.4,41580,MEGA,8.645161,6.193548,LARGE,LOW,LARGE_LOW,4,1,1,4,0,10,LARGE,0,NaN,0.0,1.0
670,264409,13,87533,2682086,13,5,게임,2,2447400,5.136868e+05,486640.0,NONE,5400,3800,2025-07-26 03:09:09,2025-08-25 11:17:13,Battle Night: Cyberpunk RPG,2025-04-03 16:00:00,0,31,0.419355,0.0,1600,0.4,20800,MEGA,86518.903226,2823.645161,MEGA,LOW,MEGA_LOW,4,0,4,4,0,12,LARGE,1,14,1.0,1.0
673,313780,10,87184,2694154,0,5,게임,2,2447700,NaN,NaN,NONE,6000,4500,2025-07-26 05:02:44,2025-08-25 11:10:27,The Farmers,2025-04-09 10:00:00,0,31,0.000000,0.0,1500,0.3,0,MEGA,86908.193548,2812.387097,MEGA,LOW,MEGA_LOW,3,0,4,4,0,11,LARGE,1,58,1.0,1.0
681,339422,14,625,1430,84,5,게임,2,2453300,1.050223e+04,4038.0,NONE,4800,3600,2025-07-26 00:50:26,2025-08-25 11:06:07,윈조이 포커,2025-04-15 10:00:00,0,31,2.709677,0.1,1200,0.3,100800,MEGA,46.129032,20.161290,MEGA,LOW,MEGA_LOW,4,1,2,4,0,11,LARGE,0,"563,58,18,14",1.0,1.0
699,360191,4,124,338,0,5,게임,1,2449800,NaN,NaN,NONE,9000,6700,2025-07-26 00:14:35,2025-08-25 08:45:02,Merge Islanders: Magic Puzzle,2025-05-08 10:00:00,0,31,0.000000,0.0,2300,0.3,0,LARGE,10.903226,4.000000,LARGE,LOW,LARGE_LOW,3,0,1,4,0,8,LARGE,0,562,1.0,1.0
700,360192,6,87048,2708758,1,5,게임,2,2449700,1.913485e+06,1913485.0,NONE,6000,4500,2025-07-26 00:26:31,2025-08-25 06:02:00,Merge Islanders: Magic Puzzle,2025-05-08 10:00:00,0,31,0.032258,0.0,1500,0.3,1500,MEGA,87379.290323,2808.000000,MEGA,LOW,MEGA_LOW,3,0,4,4,0,11,LARGE,1,58,1.0,1.0


In [17]:
ads_351170_analysis = analyze_ads_performance(351170, click)
ads_351170_analysis

,ads_idx,mda_idx,total_clicks,total_conversions,contract_price,media_price,domain,ads_category,cvr,profit_per_conversion,total_profit,first_click,last_click,days_active_calc,daily_clicks,daily_conversions,daily_profit,배분그룹
0,351170,58,13361,62,7200,5200,게임,5,0.0046,2000,124000,2025-07-26 00:09:23,2025-08-25 11:19:37,31,431.000000,2.000000,4000.000000,잘 배분
1,351170,14,438,32,7200,5200,게임,5,0.0731,2000,64000,2025-07-26 13:47:51,2025-08-25 11:17:31,30,14.600000,1.066667,2133.333333,잘 배분
2,351170,539,259,9,9600,7200,게임,5,0.0347,2400,21600,2025-08-15 09:21:46,2025-08-25 10:45:46,11,23.545455,0.818182,1963.636364,잘 배분
3,351170,18,444,21,7200,5200,게임,5,0.0473,2000,42000,2025-07-26 00:58:03,2025-08-24 19:56:06,30,14.800000,0.700000,1400.000000,잘 배분
4,351170,667,38,2,9600,7200,게임,5,0.0526,2400,4800,2025-07-26 18:56:09,2025-08-23 20:11:22,29,1.310345,0.068966,165.517241,잘 배분
5,351170,270,13,1,7200,3640,게임,5,0.0769,3560,3560,2025-07-29 17:00:24,2025-08-23 18:56:17,26,0.500000,0.038462,136.923077,잘 배분
6,351170,643,9,1,7200,5200,게임,5,0.1111,2000,2000,2025-08-06 18:03:21,2025-08-22 13:49:05,16,0.562500,0.062500,125.000000,잘 배분
7,351170,22,1,0,7200,5200,게임,5,0.0000,2000,0,2025-08-03 01:04:12,2025-08-03 01:04:12,1,1.000000,0.000000,0.000000,잘못 배분
8,351170,54,76,0,9600,7200,게임,5,0.0000,2400,0,2025-08-20 11:35:55,2025-08-24 18:42:35,5,15.200000,0.000000,0.000000,잘못 배분
9,351170,294,3,0,7200,5200,게임,5,0.0000,2000,0,2025-07-26 22:01:56,2025-08-10 20:57:22,15,0.200000,0.000000,0.000000,잘못 배분


In [18]:
ads_16528_analysis = analyze_ads_performance(16528, click)
ads_16528_analysis

,ads_idx,mda_idx,total_clicks,total_conversions,contract_price,media_price,domain,ads_category,cvr,profit_per_conversion,total_profit,first_click,last_click,days_active_calc,daily_clicks,daily_conversions,daily_profit,배분그룹
0,16528,686,1,1,2000,1600,생활,8,1.0000,400,400,2025-08-17 19:29:08,2025-08-17 19:29:08,1,1.000000,1.000000,400.000000,잘 배분
1,16528,667,512,27,2000,1600,생활,8,0.0527,400,10800,2025-07-26 02:01:55,2025-08-25 10:25:46,31,16.516129,0.870968,348.387097,잘 배분
2,16528,790,4,1,2000,1600,생활,8,0.2500,400,400,2025-07-26 23:18:34,2025-08-16 13:04:05,21,0.190476,0.047619,19.047619,잘 배분
3,16528,761,75,1,2000,1600,생활,8,0.0133,400,400,2025-07-26 12:26:24,2025-08-25 01:11:20,30,2.500000,0.033333,13.333333,잘 배분
4,16528,270,3,0,2000,1120,생활,8,0.0000,880,0,2025-07-27 21:12:35,2025-07-31 06:49:57,4,0.750000,0.000000,0.000000,잘 배분
5,16528,281,9,0,2000,1600,생활,8,0.0000,400,0,2025-07-26 18:45:46,2025-08-24 07:22:37,29,0.310345,0.000000,0.000000,잘 배분
6,16528,337,3,0,2000,1600,생활,8,0.0000,400,0,2025-08-02 21:32:23,2025-08-21 07:37:39,19,0.157895,0.000000,0.000000,잘 배분
7,16528,480,1,0,2000,1600,생활,8,0.0000,400,0,2025-08-11 00:01:52,2025-08-11 00:01:52,1,1.000000,0.000000,0.000000,잘 배분
8,16528,564,1,0,2000,1600,생활,8,0.0000,400,0,2025-08-22 07:55:34,2025-08-22 07:55:34,1,1.000000,0.000000,0.000000,잘 배분
9,16528,772,2,0,2000,1600,생활,8,0.0000,400,0,2025-08-06 14:18:18,2025-08-06 14:42:53,1,2.000000,0.000000,0.000000,잘 배분


In [19]:
ads_157236_analysis = analyze_ads_performance(157236, click)
ads_157236_analysis

,ads_idx,mda_idx,total_clicks,total_conversions,contract_price,media_price,domain,ads_category,cvr,profit_per_conversion,total_profit,first_click,last_click,days_active_calc,daily_clicks,daily_conversions,daily_profit,배분그룹
0,157236,563,156,66,30,20,식당/카페,4,0.4231,10,660,2025-07-26 14:33:10,2025-08-24 12:49:59,29,5.37931,2.275862,22.758621,잘 배분
1,157236,562,2,1,30,20,식당/카페,4,0.5000,10,10,2025-08-06 19:01:55,2025-08-06 19:02:30,1,2.00000,1.000000,10.000000,잘 배분
2,157236,686,1,1,30,20,식당/카페,4,1.0000,10,10,2025-08-13 14:34:50,2025-08-13 14:34:50,1,1.00000,1.000000,10.000000,잘 배분
3,157236,790,1,1,30,20,식당/카페,4,1.0000,10,10,2025-07-30 16:56:34,2025-07-30 16:56:34,1,1.00000,1.000000,10.000000,잘 배분
4,157236,854,3,2,30,20,식당/카페,4,0.6667,10,20,2025-08-12 13:47:38,2025-08-19 21:54:12,8,0.37500,0.250000,2.500000,잘못 배분
5,157236,817,3,1,30,20,식당/카페,4,0.3333,10,10,2025-07-31 15:05:02,2025-08-24 10:33:00,24,0.12500,0.041667,0.416667,잘못 배분
6,157236,772,1,0,30,20,식당/카페,4,0.0000,10,0,2025-08-23 23:38:24,2025-08-23 23:38:24,1,1.00000,0.000000,0.000000,잘못 배분


### 게임 도메인

In [43]:
ads_pool[(ads_pool) & (ads_pool['domain'] == '게임')]

,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,ads_order,ctit_mean,ctit_median,ads_rejoin_type,contract_price,media_price,first_click,last_click,ads_name,ads_sdate,expire,days_active,daily_avg_conversions,cvr,margin,roi,total_net_return,ads_level_detailed,daily_clicks,daily_users,scale_level,performance_level,ads_level,media_score,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster,mda_idx_arr,M,A
8,10500,2,8,10,0,5,게임,2,2414800,NaN,NaN,NONE,4000,3000,2025-07-29 05:39:08,2025-08-24 10:13:02,판타지타운,2021-03-19 18:00:00,0,27,0.000000,0.0,1000,0.3,0,MEDIUM,0.370370,0.296296,MEDIUM,LOW,MEDIUM_LOW,2,0,0,3,0,5,MEDIUM,0,NaN,0.0,1.0
24,11059,9,111,260,7,5,게임,2,2097400,5449.571429,1817.0,NONE,300,200,2025-07-26 11:40:34,2025-08-25 00:14:56,파이널삼국지2,2021-05-03 17:00:00,0,30,0.233333,0.0,100,0.5,700,LARGE,8.666667,3.700000,LARGE,LOW,LARGE_LOW,3,0,1,3,0,7,MEDIUM,0,NaN,0.0,1.0
25,11123,1,12,13,8,6,게임,2,1429100,36649.250000,19320.5,NONE,0,300,2025-07-28 17:40:19,2025-08-21 00:29:48,프린세스테일,2021-05-10 14:00:00,0,24,0.333333,0.6,-300,-1.0,-2400,MICRO,0.541667,0.500000,SMALL,LOW,SMALL_LOW,1,0,0,3,1,5,MEDIUM,0,NaN,0.0,1.0
38,16485,1,43,99,11,6,게임,2,1773300,158545.272727,64260.0,NONE,0,475,2025-07-26 00:49:55,2025-08-24 12:56:15,브롤스타즈,2022-04-18 19:00:00,0,30,0.366667,0.1,-475,-1.0,-5225,MEDIUM,3.300000,1.433333,SMALL,LOW,SMALL_LOW,1,0,1,3,0,5,MEDIUM,0,NaN,0.0,1.0
49,17234,1,16,39,8,0,게임,2,1823000,137977.125000,120622.0,NONE,0,399,2025-07-26 15:26:49,2025-08-24 11:55:17,IDLE ANGELS : 여신전쟁,2022-05-20 17:00:00,0,29,0.275862,0.2,-399,-1.0,-3192,SMALL,1.344828,0.551724,SMALL,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0,NaN,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4439,446087,1,4,6,0,5,게임,2,2467800,NaN,NaN,NONE,15000,7875,2025-08-21 14:28:48,2025-08-23 13:00:43,흔직세,2025-08-21 14:00:00,0,2,0.000000,0.0,7125,0.9,0,LARGE,3.000000,2.000000,SMALL,LOW,SMALL_LOW,1,0,1,1,0,3,SMALL,0,NaN,0.0,1.0
4440,446088,1,15,53,0,5,게임,2,2470800,NaN,NaN,NONE,15000,7875,2025-08-21 14:07:49,2025-08-25 10:07:06,네기마,2025-08-21 13:00:00,0,4,0.000000,0.0,7125,0.9,0,LARGE,13.250000,3.750000,SMALL,LOW,SMALL_LOW,1,0,1,1,0,3,SMALL,0,NaN,0.0,1.0
4447,446111,2,2,2,2,4,게임,7,1369936,33.000000,33.0,NONE,15,10,2025-08-22 00:13:16,2025-08-24 04:40:02,인형뽑기기계,2025-08-21 15:27:20,0,3,0.666667,1.0,5,0.5,10,MICRO,0.666667,0.666667,MEDIUM,HIGH,MEDIUM_HIGH,2,0,0,1,4,7,MEDIUM,0,"562,563",1.0,1.0
4463,446139,1,1,1,1,4,게임,7,1369936,33.000000,33.0,NONE,15,10,2025-08-23 00:32:37,2025-08-23 00:32:37,출차주의등,2025-08-22 00:09:33,0,1,1.000000,1.0,5,0.5,5,MICRO,1.000000,1.000000,SMALL,HIGH,SMALL_HIGH,1,1,0,0,4,6,MEDIUM,0,"562,563",1.0,1.0


In [24]:
# 게임 - 264408
ads_264408_analysis = analyze_ads_performance(264408, click)
ads_264408_analysis

,ads_idx,mda_idx,total_clicks,total_conversions,contract_price,media_price,domain,ads_category,cvr,profit_per_conversion,total_profit,first_click,last_click,days_active_calc,daily_clicks,daily_conversions,daily_profit,배분그룹
0,264408,643,62,39,2040,1500,게임,5,0.6290,540,21060,2025-07-26 01:30:51,2025-08-25 02:28:02,31,2.000000,1.258065,679.354839,잘 배분
1,264408,645,31,21,2040,1500,게임,5,0.6774,540,11340,2025-07-26 06:39:35,2025-08-25 09:09:34,31,1.000000,0.677419,365.806452,잘 배분
2,264408,634,7,7,2040,1500,게임,5,1.0000,540,3780,2025-08-04 00:54:56,2025-08-22 12:34:49,19,0.368421,0.368421,198.947368,잘 배분
3,264408,270,29,3,2040,1050,게임,5,0.1034,990,2970,2025-07-30 15:38:39,2025-08-20 02:44:55,21,1.380952,0.142857,141.428571,잘 배분
4,264408,667,19,1,2040,1500,게임,5,0.0526,540,540,2025-07-26 01:47:12,2025-07-29 07:25:42,4,4.750000,0.250000,135.000000,잘 배분
5,264408,761,93,4,2040,1500,게임,5,0.0430,540,2160,2025-07-26 16:28:08,2025-08-25 08:07:13,30,3.100000,0.133333,72.000000,잘 배분
6,264408,790,4,1,2040,1500,게임,5,0.2500,540,540,2025-07-29 05:15:39,2025-08-07 00:55:10,9,0.444444,0.111111,60.000000,잘못 배분
7,264408,281,7,1,2040,1500,게임,5,0.1429,540,540,2025-07-27 00:03:45,2025-08-17 12:13:57,22,0.318182,0.045455,24.545455,잘못 배분
8,264408,108,4,0,2040,1500,게임,5,0.0000,540,0,2025-08-07 20:36:49,2025-08-07 20:37:13,1,4.000000,0.000000,0.000000,잘못 배분
9,264408,337,5,0,2040,1500,게임,5,0.0000,540,0,2025-08-03 23:07:45,2025-08-23 00:44:48,20,0.250000,0.000000,0.000000,잘못 배분


In [37]:
media_portfolio[media_portfolio['mda_idx'].isin(ads_264408_analysis['mda_idx'].unique())]

,mda_idx,user_count,total_clicks,total_conversions,first_click,last_click,days_active,daily_avg_conversions,LARGE,MEDIUM,MEGA,SMALL,total_ads,MEGA_ratio,LARGE_ratio,MEDIUM_ratio,SMALL_ratio,ads_category_0_pct,ads_category_1_pct,ads_category_2_pct,ads_category_3_pct,ads_category_4_pct,ads_category_5_pct,ads_category_6_pct,ads_category_7_pct,...,ads_category_13_pct,ads_os_type_1_pct,ads_os_type_2_pct,ads_os_type_3_pct,ads_os_type_7_pct,domain_게임_pct,domain_교육_pct,domain_금융_pct,domain_기타_pct,domain_미디어/컨텐츠_pct,domain_뷰티_pct,domain_비영리/공공_pct,domain_생활_pct,domain_식당/카페_pct,domain_식음료_pct,domain_운동/스포츠_pct,domain_운세_pct,domain_의료/건강_pct,domain_채용_pct,domain_커머스_pct,expected_total_profit,conversion_rate,last_click_dt,classification,basic_classification
12,108,676,813,666,2025-08-01 15:52:47,2025-08-25 05:12:35,24,27.750000,12,21,8,4,45,17.8,26.7,46.7,8.9,4.444444,2.222222,8.888889,0.000000,0.000000,11.111111,0.000000,2.222222,...,0.000000,0.000000,33.333333,0.000000,66.666667,22.222222,0.000000,15.555556,0.000000,6.666667,20.000000,0.000000,2.222222,0.000000,13.333333,0.000000,8.888889,2.222222,0.000000,8.888889,76180,0.819188,2025-08-25 05:12:35,안정공급형(후보),안정공급형(후보)
14,270,3146,61793,29548,2025-07-26 00:01:09,2025-08-25 11:23:53,31,953.161290,782,677,74,268,1801,4.1,43.4,37.6,14.9,2.165464,1.277068,2.054414,0.888395,68.739589,7.717934,4.775125,1.388118,...,0.555247,0.000000,19.822321,0.555247,79.622432,15.935591,0.666297,4.330927,4.997224,2.831760,5.052748,0.333148,12.770683,12.715158,3.997779,2.776235,0.943920,6.496391,0.333148,25.818989,-144697,0.478177,2025-08-25 11:23:53,대량처리형,대량처리형
15,281,314,1442,129,2025-07-26 00:46:27,2025-08-25 10:02:45,31,4.161290,60,106,20,8,194,10.3,30.9,54.6,4.1,1.030928,4.639175,2.061856,3.092784,0.000000,29.896907,0.000000,8.247423,...,0.000000,0.000000,51.030928,0.000000,48.969072,30.927835,0.000000,24.226804,0.515464,10.309278,3.092784,1.546392,7.731959,0.000000,6.185567,1.030928,1.546392,2.061856,1.030928,9.793814,24762,0.089459,2025-08-25 10:02:45,안정공급형(후보),안정공급형(후보)
21,337,603,1920,292,2025-07-26 00:21:49,2025-08-25 09:11:51,31,9.419355,49,245,19,69,382,5.0,12.8,64.1,18.1,1.047120,0.785340,2.356021,1.047120,0.000000,25.654450,0.000000,2.879581,...,0.000000,0.000000,34.293194,0.261780,65.445026,28.534031,0.261780,9.162304,0.261780,3.403141,10.732984,0.261780,2.879581,0.261780,6.282723,1.832461,1.832461,9.424084,0.523560,24.345550,164850,0.152083,2025-08-25 09:11:51,안정공급형(후보),안정공급형(후보)
44,482,1559,3624,1647,2025-07-26 07:58:59,2025-08-25 10:38:07,31,53.129032,22,10,21,1,54,38.9,40.7,18.5,1.9,0.000000,14.814815,7.407407,7.407407,0.000000,11.111111,0.000000,5.555556,...,7.407407,0.000000,48.148148,0.000000,51.851852,22.222222,0.000000,25.925926,1.851852,14.814815,7.407407,0.000000,0.000000,0.000000,1.851852,1.851852,5.555556,0.000000,3.703704,14.814815,93593,0.454470,2025-08-25 10:38:07,안정공급형(후보),안정공급형(후보)
59,634,134,5271,3321,2025-07-26 01:47:57,2025-08-25 08:57:59,31,107.129032,902,1028,25,496,2451,1.0,36.8,41.9,20.2,0.203998,0.203998,0.448796,0.081599,97.796818,0.326397,0.000000,0.122399,...,0.163199,0.000000,1.101591,0.040800,98.857609,1.509588,0.693594,1.264790,11.301510,1.264790,5.059160,0.326397,13.667891,20.277438,6.813545,3.712770,0.203998,12.566299,0.040800,21.297430,41638,0.630051,2025-08-25 08:57:59,안정공급형(후보),안정공급형(후보)
63,643,294,983,265,2025-07-26 00:19:27,2025-08-25 05:19:19,31,8.548387,25,46,19,4,94,20.2,26.6,48.9,4.3,0.000000,0.000000,21.276596,0.000000,0.000000,78.723404,0.000000,0.000000,...,0.000000,0.000000,89.361702,6.382979,4.255319,91.489362,0.000000,0.000000,0.000000,1.063830,0.000000,1.063830,0.000000,1.063830,3.191489,1.063830,0.000000,0.000000,1.063830,0.000000,95536,0.269583,2025-08-25 05:19:19,특화전문형_게임특화,특화전문형
64,645,369,1050,627,2025-07-26 00:22:43,2025-08-25 10:38:53,31,20.225806,80,45,37,16,178,20.8,44.9,25.3,9.0,3.370787,7.865169,9.550562,2.808989,46.067416,3.932584,0.000000,1.685393,...,3.932584,0.000000,26.966292,2.247191,70.786517,13.483146,0.561798,11.235955,0.561

In [39]:
media_portfolio[media_portfolio['mda_idx'].isin([14, 1027, 496])]

,mda_idx,user_count,total_clicks,total_conversions,first_click,last_click,days_active,daily_avg_conversions,LARGE,MEDIUM,MEGA,SMALL,total_ads,MEGA_ratio,LARGE_ratio,MEDIUM_ratio,SMALL_ratio,ads_category_0_pct,ads_category_1_pct,ads_category_2_pct,ads_category_3_pct,ads_category_4_pct,ads_category_5_pct,ads_category_6_pct,ads_category_7_pct,...,ads_category_13_pct,ads_os_type_1_pct,ads_os_type_2_pct,ads_os_type_3_pct,ads_os_type_7_pct,domain_게임_pct,domain_교육_pct,domain_금융_pct,domain_기타_pct,domain_미디어/컨텐츠_pct,domain_뷰티_pct,domain_비영리/공공_pct,domain_생활_pct,domain_식당/카페_pct,domain_식음료_pct,domain_운동/스포츠_pct,domain_운세_pct,domain_의료/건강_pct,domain_채용_pct,domain_커머스_pct,expected_total_profit,conversion_rate,last_click_dt,classification,basic_classification
1,14,31834,84763,22983,2025-07-26 00:01:22,2025-08-25 11:22:40,31,741.387097,31,5,22,1,59,37.3,52.5,8.5,1.7,0.000000,20.338983,18.644068,8.474576,0.0,25.423729,0.0,3.389831,...,0.0,42.372881,38.983051,0.000000,18.644068,44.067797,0.000000,11.864407,3.389831,8.474576,5.084746,0.000000,13.559322,0.00,3.389831,0.0,0.000000,3.389831,1.694915,5.084746,2733547,0.271144,2025-08-25 11:22:40,안정공급형,안정공급형
46,496,80048,127975,26418,2025-07-26 00:06:05,2025-08-25 11:24:01,31,852.193548,24,8,22,0,54,40.7,44.4,14.8,0.0,16.666667,11.111111,20.370370,11.111111,0.0,3.703704,0.0,9.259259,...,0.0,0.000000,66.666667,7.407407,25.925926,25.925926,1.851852,25.925926,1.851852,7.407407,1.851852,5.555556,9.259259,0.00,5.555556,0.0,1.851852,3.703704,1.851852,7.407407,1639389,0.206431,2025-08-25 11:24:01,안정공급형,안정공급형
165,1027,1521,2057,1001,2025-07-26 00:03:07,2025-08-25 11:18:54,31,32.290323,1,0,15,0,16,93.8,6.2,0.0,0.0,0.000000,31.250000,50.000000,18.750000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,81.250000,0.000000,18.750000,31.250000,0.000000,6.250000,0.000000,12.500000,6.250000,6.250000,12.500000,6.25,0.000000,0.0,0.000000,0.000000,6.250000,12.500000,76490,0.486631,2025-08-25 11:18:54,특화전문형_MEGA특화,특화전문형


### 금융 - 창민님 거

In [27]:
ads_22232_analysis = analyze_ads_performance(22232, click)
ads_22232_analysis

,ads_idx,mda_idx,total_clicks,total_conversions,contract_price,media_price,domain,ads_category,cvr,profit_per_conversion,total_profit,first_click,last_click,days_active_calc,daily_clicks,daily_conversions,daily_profit,배분그룹
0,22232,398,731,130,800,640,금융,8,0.1778,160,20800,2025-07-26 00:46:23,2025-08-25 09:01:33,31,23.580645,4.193548,670.967742,잘 배분
1,22232,496,316,92,800,640,금융,8,0.2911,160,14720,2025-07-26 02:23:32,2025-08-25 07:45:37,31,10.193548,2.967742,474.838710,잘 배분
2,22232,54,159,59,800,640,금융,8,0.3711,160,9440,2025-07-26 03:54:05,2025-08-25 10:05:25,31,5.129032,1.903226,304.516129,잘 배분
3,22232,761,105,19,800,640,금융,8,0.1810,160,3040,2025-07-26 15:11:31,2025-08-24 14:44:58,29,3.620690,0.655172,104.827586,잘 배분
4,22232,270,5,3,800,448,금융,8,0.6000,352,1056,2025-07-27 09:19:16,2025-08-20 07:38:46,24,0.208333,0.125000,44.000000,잘못 배분
5,22232,790,7,2,800,640,금융,8,0.2857,160,320,2025-08-04 01:09:14,2025-08-23 01:18:03,20,0.350000,0.100000,16.000000,잘못 배분
6,22232,30,1,0,800,640,금융,8,0.0000,160,0,2025-08-18 13:06:29,2025-08-18 13:06:29,1,1.000000,0.000000,0.000000,잘못 배분


In [31]:
ads_22232_analysis['mda_idx'].unique()

array([398, 496,  54, 761, 270, 790,  30])

In [32]:
media_portfolio[media_portfolio['mda_idx'].isin((ads_22232_analysis['mda_idx'].unique()))]

,mda_idx,user_count,total_clicks,total_conversions,first_click,last_click,days_active,daily_avg_conversions,LARGE,MEDIUM,MEGA,SMALL,total_ads,MEGA_ratio,LARGE_ratio,MEDIUM_ratio,SMALL_ratio,ads_category_0_pct,ads_category_1_pct,ads_category_2_pct,ads_category_3_pct,ads_category_4_pct,ads_category_5_pct,ads_category_6_pct,ads_category_7_pct,...,ads_category_13_pct,ads_os_type_1_pct,ads_os_type_2_pct,ads_os_type_3_pct,ads_os_type_7_pct,domain_게임_pct,domain_교육_pct,domain_금융_pct,domain_기타_pct,domain_미디어/컨텐츠_pct,domain_뷰티_pct,domain_비영리/공공_pct,domain_생활_pct,domain_식당/카페_pct,domain_식음료_pct,domain_운동/스포츠_pct,domain_운세_pct,domain_의료/건강_pct,domain_채용_pct,domain_커머스_pct,expected_total_profit,conversion_rate,last_click_dt,classification,basic_classification
5,30,3606,5160,391,2025-07-26 00:42:21,2025-08-25 11:06:44,31,12.612903,14,37,9,0,60,15.0,23.3,61.7,0.0,0.000000,5.000000,1.666667,8.333333,0.000000,38.333333,0.000000,3.333333,...,0.000000,0.000000,58.333333,0.000000,41.666667,41.666667,1.666667,18.333333,0.000000,11.666667,1.666667,3.333333,8.333333,0.000000,1.666667,0.000000,3.333333,1.666667,3.333333,3.333333,22520,0.075775,2025-08-25 11:06:44,안정공급형(후보),안정공급형(후보)
7,54,6395,12838,4299,2025-07-26 00:12:15,2025-08-25 11:11:45,31,138.677419,69,172,29,61,331,8.8,20.8,52.0,18.4,2.416918,4.833837,4.229607,3.021148,4.229607,12.084592,0.000000,3.021148,...,1.208459,11.178248,18.429003,0.302115,70.090634,17.824773,0.302115,12.688822,0.000000,3.625378,10.574018,1.208459,4.229607,1.208459,7.250755,1.510574,4.229607,9.667674,0.906344,24.773414,309633,0.334865,2025-08-25 11:11:45,안정공급형(후보),안정공급형(후보)
14,270,3146,61793,29548,2025-07-26 00:01:09,2025-08-25 11:23:53,31,953.161290,782,677,74,268,1801,4.1,43.4,37.6,14.9,2.165464,1.277068,2.054414,0.888395,68.739589,7.717934,4.775125,1.388118,...,0.555247,0.000000,19.822321,0.555247,79.622432,15.935591,0.666297,4.330927,4.997224,2.831760,5.052748,0.333148,12.770683,12.715158,3.997779,2.776235,0.943920,6.496391,0.333148,25.818989,-144697,0.478177,2025-08-25 11:23:53,대량처리형,대량처리형
36,398,3267,6945,1185,2025-07-26 00:17:33,2025-08-25 09:57:19,31,38.225806,7,6,14,0,27,51.9,25.9,22.2,0.0,0.000000,11.111111,18.518519,18.518519,0.000000,3.703704,0.000000,18.518519,...,0.000000,0.000000,48.148148,3.703704,48.148148,25.925926,3.703704,33.333333,0.000000,7.407407,0.000000,11.111111,3.703704,0.000000,7.407407,0.000000,0.000000,3.703704,0.000000,3.703704,185580,0.170626,2025-08-25 09:57:19,안정공급형(후보),안정공급형(후보)
46,496,80048,127975,26418,2025-07-26 00:06:05,2025-08-25 11:24:01,31,852.193548,24,8,22,0,54,40.7,44.4,14.8,0.0,16.666667,11.111111,20.370370,11.111111,0.000000,3.703704,0.000000,9.259259,...,0.000000,0.000000,66.666667,7.407407,25.925926,25.925926,1.851852,25.925926,1.851852,7.407407,1.851852,5.555556,9.259259,0.000000,5.555556,0.000000,1.851852,3.703704,1.851852,7.407407,1639389,0.206431,2025-08-25 11:24:01,안정공급형,안정공급형
105,761,9314,71853,13419,2025-07-26 00:13:29,2025-08-25 11:17:02,31,432.870968,126,200,42,40,408,10.3,30.9,49.0,9.8,3.431373,3.186275,5.637255,2.450980,20.098039,33.578431,0.000000,3.186275,...,0.490196,12.254902,42.647059,0.980392,44.117647,38.480392,0.735294,14.705882,0.000000,5.392157,2.450980,0.735294,13.480392,8.333333,3.431373,0.980392,3.676471,3.921569,0.735294,2.941176,781705,0.186756,2025-08-25 11:17:02,안정공급형,안정공급형
113,790,724,1447,742,2025-07-26 00:28:49,2025-08-25 09:04:27,31,23.935484,80,47,25,6,158,15.8,50.6,29.7,3.8,0.632911,3.164557,10.759494,4.430380,11.392405,16.455696,0.000000,5.063291,...,0.000000,0.000000,38.607595,3.164557,58.227848,25.949367,0.632911,21.518987,0.000000,13.291139,2.531646,2.531646,11.392405,3.164557,3.164557,1.265823,3.797468,6.329114,0.632911,3.797468,254068,0.512785,2025-08-25 09:04:27,안정공급형(후보),안정공급형(후보)


In [36]:
media_portfolio[media_portfolio['mda_idx'].isin([14,22, 396, 343, 990])]

,mda_idx,user_count,total_clicks,total_conversions,first_click,last_click,days_active,daily_avg_conversions,LARGE,MEDIUM,MEGA,SMALL,total_ads,MEGA_ratio,LARGE_ratio,MEDIUM_ratio,SMALL_ratio,ads_category_0_pct,ads_category_1_pct,ads_category_2_pct,ads_category_3_pct,ads_category_4_pct,ads_category_5_pct,ads_category_6_pct,ads_category_7_pct,...,ads_category_13_pct,ads_os_type_1_pct,ads_os_type_2_pct,ads_os_type_3_pct,ads_os_type_7_pct,domain_게임_pct,domain_교육_pct,domain_금융_pct,domain_기타_pct,domain_미디어/컨텐츠_pct,domain_뷰티_pct,domain_비영리/공공_pct,domain_생활_pct,domain_식당/카페_pct,domain_식음료_pct,domain_운동/스포츠_pct,domain_운세_pct,domain_의료/건강_pct,domain_채용_pct,domain_커머스_pct,expected_total_profit,conversion_rate,last_click_dt,classification,basic_classification
1,14,31834,84763,22983,2025-07-26 00:01:22,2025-08-25 11:22:40,31,741.387097,31,5,22,1,59,37.3,52.5,8.5,1.7,0.000000,20.338983,18.644068,8.474576,0.000000,25.423729,0.0,3.389831,...,0.000000,42.372881,38.983051,0.000000,18.644068,44.067797,0.00000,11.864407,3.389831,8.474576,5.084746,0.000000,13.559322,0.000000,3.389831,0.000000,0.000000,3.389831,1.694915,5.084746,2733547,0.271144,2025-08-25 11:22:40,안정공급형,안정공급형
3,22,37668,83355,35232,2025-07-26 00:00:31,2025-08-25 11:14:50,31,1136.516129,67,58,33,8,166,19.9,40.4,34.9,4.8,8.433735,6.626506,10.843373,5.421687,0.000000,10.240964,0.0,6.626506,...,0.000000,0.000000,40.361446,3.012048,56.626506,19.277108,0.60241,26.506024,0.000000,10.240964,1.807229,3.012048,11.445783,0.602410,4.216867,1.204819,7.831325,4.819277,1.204819,7.228916,2541770,0.422674,2025-08-25 11:14:50,대량처리형,대량처리형
25,343,75814,181809,90504,2025-07-26 00:34:21,2025-08-25 11:21:25,31,2919.483871,6,3,8,0,17,47.1,35.3,17.6,0.0,0.000000,0.000000,47.058824,0.000000,0.000000,5.882353,0.0,0.000000,...,0.000000,23.529412,41.176471,0.000000,35.294118,47.058824,0.00000,0.000000,5.882353,0.000000,35.294118,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.764706,3931080,0.497797,2025-08-25 11:21:25,안정공급형(후보),안정공급형(후보)
34,396,50362,157005,90729,2025-07-29 16:54:41,2025-08-25 10:51:14,27,3360.333333,1,1,4,0,6,66.7,16.7,16.7,0.0,0.000000,0.000000,83.333333,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,100.000000,0.000000,0.000000,66.666667,0.00000,0.000000,16.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,16.666667,3692160,0.577873,2025-08-25 10:51:14,특화전문형_게임특화,특화전문형
146,990,2533,4366,1573,2025-07-26 00:09:59,2025-08-25 11:22:27,31,50.741935,3,1,14,0,18,77.8,16.7,5.6,0.0,0.000000,11.111111,44.444444,22.222222,5.555556,5.555556,0.0,0.000000,...,5.555556,0.000000,66.666667,0.000000,33.333333,38.888889,0.00000,0.000000,0.000000,11.111111,0.000000,0.000000,5.555556,5.555556,11.111111,0.000000,0.000000,0.000000,5.555556,22.222222,65362,0.360284,2025-08-25 11:22:27,특화전문형_MEGA특화,특화전문형


In [50]:
# ads_pool
ads_pool[ads_pool.first_click >= '2025-08-22 00:00:00']

,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,ads_order,ctit_mean,ctit_median,ads_rejoin_type,contract_price,media_price,first_click,last_click,ads_name,ads_sdate,expire,days_active,daily_avg_conversions,cvr,margin,roi,total_net_return,ads_level_detailed,daily_clicks,daily_users,scale_level,performance_level,ads_level,media_score,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster,mda_idx_arr,M,A
81,20889,1,1,1,0,11,의료/건강,7,1313118,NaN,NaN,REJOINABLE,20454,13200,2025-08-22 04:25:52,2025-08-22 04:25:52,보령 콘드로이친 칼마D(800mgx90정),2023-02-13 13:00:00,0,1,0.0,0.0,7254,0.5,0,LARGE,1.0,1.0,SMALL,LOW,SMALL_LOW,1,0,0,0,0,1,SMALL,0,NaN,0.0,1.0
91,20914,1,2,3,0,11,뷰티,7,1248052,NaN,NaN,REJOINABLE,37272,24200,2025-08-22 08:07:56,2025-08-23 04:49:26,셀린온 바이오 가드 리프팅 스킨케어 3종,2023-02-13 13:00:00,0,1,0.0,0.0,13072,0.5,0,LARGE,3.0,2.0,SMALL,LOW,SMALL_LOW,1,0,1,0,0,2,SMALL,0,NaN,0.0,1.0
161,27829,1,1,1,0,11,커머스,7,1107644,NaN,NaN,REJOINABLE,8909,5700,2025-08-24 10:30:04,2025-08-24 10:30:04,아크로 BUDS ME 블루투스 이어폰 흰색,2024-01-08 10:00:00,0,1,0.0,0.0,3209,0.6,0,LARGE,1.0,1.0,SMALL,LOW,SMALL_LOW,1,0,0,0,0,1,SMALL,0,NaN,0.0,1.0
166,27879,1,1,1,0,11,의료/건강,7,1313323,NaN,NaN,REJOINABLE,20909,13500,2025-08-24 14:12:27,2025-08-24 14:12:27,대현헬스케어 눈건강 루테인지아잔틴,2024-01-10 09:00:00,0,1,0.0,0.0,7409,0.5,0,LARGE,1.0,1.0,SMALL,LOW,SMALL_LOW,1,0,0,0,0,1,SMALL,0,NaN,0.0,1.0
170,27898,1,1,1,0,11,커머스,7,1362820,NaN,NaN,REJOINABLE,12727,8200,2025-08-22 22:39:29,2025-08-22 22:39:29,"베스트하임 UV 안심 변기 살균기,자동살균",2024-01-11 10:00:00,0,1,0.0,0.0,4527,0.6,0,LARGE,1.0,1.0,SMALL,LOW,SMALL_LOW,1,0,0,0,0,1,SMALL,0,NaN,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4709,446899,2,52,63,49,4,커머스,7,1369916,42.102041,39.0,ADS_CODE_DAILY_UPDATE,18,14,2025-08-25 09:40:45,2025-08-25 09:58:56,원스토몰 180212,2025-08-25 09:37:27,0,1,49.0,0.8,4,0.3,196,SMALL,63.0,52.0,LARGE,HIGH,LARGE_HIGH,2,3,2,0,2,9,LARGE,0,NaN,0.0,0.0
4710,446900,2,44,59,41,4,식당/카페,7,1369915,48.512195,47.0,ADS_CODE_DAILY_UPDATE,18,14,2025-08-25 09:45:41,2025-08-25 09:56:38,서빙카트 식당 업소용 주방 서빙카 다용도 서빙카트 소형 2단 기본고무바퀴 180213,2025-08-25 09:42:08,0,1,41.0,0.7,4,0.3,164,SMALL,59.0,44.0,LARGE,HIGH,LARGE_HIGH,2,3,2,0,1,8,LARGE,0,NaN,0.0,0.0
4711,446902,1,2,2,0,8,미디어/컨텐츠,7,2735900,NaN,NaN,NONE,200,105,2025-08-25 10:37:49,2025-08-25 11:11:29,[네이버카페가입인사] 대장TV,2025-08-25 10:00:00,0,1,0.0,0.0,95,0.9,0,MEDIUM,2.0,2.0,SMALL,LOW,SMALL_LOW,1,0,0,0,0,1,SMALL,0,NaN,0.0,0.0
4712,446908,1,1,1,0,0,금융,2,1682806,NaN,NaN,NONE,230,180,2025-08-25 10:47:48,2025-08-25 10:47:48,NOL,2025-08-25 10:37:11,0,1,0.0,0.0,50,0.3,0,MEDIUM,1.0,1.0,SMALL,LOW,SMALL_LOW,1,0,0,0,0,1,SMALL,0,NaN,0.0,0.0


In [49]:
analyze_ads_performance(20889, click)

,ads_idx,mda_idx,total_clicks,total_conversions,contract_price,media_price,domain,ads_category,cvr,profit_per_conversion,total_profit,first_click,last_click,days_active_calc,daily_clicks,daily_conversions,daily_profit,배분그룹
0,20889,714,1,0,20454,13200,의료/건강,11,0.0,7254,0,2025-08-22 04:25:52,2025-08-22 04:25:52,1,1.0,0.0,0.0,분류불가


In [67]:
ad1 = [np.int64(446464), np.int64(446465), np.int64(446466), np.int64(446469), np.int64(446470), np.int64(446472), np.int64(446475), np.int64(446476), np.int64(446478), np.int64(28686), np.int64(446480), np.int64(446482), np.int64(446483), np.int64(446484), np.int64(446486), np.int64(446489), np.int64(446491), np.int64(446492), np.int64(446493), np.int64(446494), np.int64(446495), np.int64(446496), np.int64(446497), np.int64(446498), np.int64(446499), np.int64(446500), np.int64(446501), np.int64(446502), np.int64(446503), np.int64(446504), np.int64(446505), np.int64(446506), np.int64(446507), np.int64(446508), np.int64(446509), np.int64(446510), np.int64(446512), np.int64(446513), np.int64(446514), np.int64(446515), np.int64(446517), np.int64(446518), np.int64(446519), np.int64(446521), np.int64(446525), np.int64(444479), np.int64(446544), np.int64(444499), np.int64(446547), np.int64(446548), np.int64(446564), np.int64(446565), np.int64(446567), np.int64(446568), np.int64(446570), np.int64(446579), np.int64(446580), np.int64(446582), np.int64(446583), np.int64(446585), np.int64(446586), np.int64(446587), np.int64(446588), np.int64(446589), np.int64(446590), np.int64(446591), np.int64(446592), np.int64(446593), np.int64(446596), np.int64(446598), np.int64(446599), np.int64(446600), np.int64(446605), np.int64(446607), np.int64(446611), np.int64(446613), np.int64(446622), np.int64(446623), np.int64(444577), np.int64(446630), np.int64(446631), np.int64(446632), np.int64(446633), np.int64(446634), np.int64(446635), np.int64(446636), np.int64(446638), np.int64(446654), np.int64(446668), np.int64(446671), np.int64(446672), np.int64(446673), np.int64(446674), np.int64(446675), np.int64(446676), np.int64(446678), np.int64(446687), np.int64(446688), np.int64(446689), np.int64(446690), np.int64(446695), np.int64(446696), np.int64(446697), np.int64(446698), np.int64(446699), np.int64(446700), np.int64(446701), np.int64(446702), np.int64(446703), np.int64(446704), np.int64(446705), np.int64(446706), np.int64(446707), np.int64(446708), np.int64(446709), np.int64(446710), np.int64(446711), np.int64(446713), np.int64(446714), np.int64(446715), np.int64(446716), np.int64(446717), np.int64(446718), np.int64(446719), np.int64(446720), np.int64(446721), np.int64(446722), np.int64(446723), np.int64(446724), np.int64(446725), np.int64(446727), np.int64(446728), np.int64(446729), np.int64(446731), np.int64(446737), np.int64(446739), np.int64(446740), np.int64(446741), np.int64(446742), np.int64(446743), np.int64(446744), np.int64(446755), np.int64(446757), np.int64(446759), np.int64(446760), np.int64(446761), np.int64(446762), np.int64(446764), np.int64(446765), np.int64(446767), np.int64(446768), np.int64(446769), np.int64(446771), np.int64(446773), np.int64(446774), np.int64(446778), np.int64(446782), np.int64(446786), np.int64(446788), np.int64(446789), np.int64(446792), np.int64(446796), np.int64(446797), np.int64(446798), np.int64(446799), np.int64(446806), np.int64(446807), np.int64(446808), np.int64(446809), np.int64(446810), np.int64(53595), np.int64(446811), np.int64(446812), np.int64(446819), np.int64(446820), np.int64(51557), np.int64(446824), np.int64(446833), np.int64(446834), np.int64(446835), np.int64(446836), np.int64(446837), np.int64(446838), np.int64(446839), np.int64(51576), np.int64(446840), np.int64(446841), np.int64(72057), np.int64(20889), np.int64(20914), np.int64(444933), np.int64(445029), np.int64(445031), np.int64(445033), np.int64(445037), np.int64(445044), np.int64(445072), np.int64(445073), np.int64(445074), np.int64(445075), np.int64(445087), np.int64(445097), np.int64(445098), np.int64(445108), np.int64(445112), np.int64(445124), np.int64(445131), np.int64(174805), np.int64(174808), np.int64(445202), np.int64(445203), np.int64(441107), np.int64(394085), np.int64(445311), np.int64(445314), np.int64(414595), np.int64(439558), np.int64(439192), np.int64(445359), np.int64(445360), np.int64(445361), np.int64(445450), np.int64(445451), np.int64(445453), np.int64(445454), np.int64(445459), np.int64(445471), np.int64(445472), np.int64(445474), np.int64(445481), np.int64(445482), np.int64(445483), np.int64(445484), np.int64(445485), np.int64(445486), np.int64(445487), np.int64(445488), np.int64(445489), np.int64(445491), np.int64(445492), np.int64(445493), np.int64(445494), np.int64(445495), np.int64(445496), np.int64(445497), np.int64(445498), np.int64(445499), np.int64(445500), np.int64(445501), np.int64(445502), np.int64(445503), np.int64(445504), np.int64(445507), np.int64(445508), np.int64(445509), np.int64(445510), np.int64(445511), np.int64(445512), np.int64(445513), np.int64(445557), np.int64(445558), np.int64(445561), np.int64(445566), np.int64(445568), np.int64(445577), np.int64(27829), np.int64(445637), np.int64(445657), np.int64(27879), np.int64(27898), np.int64(445701), np.int64(445702), np.int64(445703), np.int64(445704), np.int64(445705), np.int64(445706), np.int64(445707), np.int64(445708), np.int64(445709), np.int64(445710), np.int64(445711), np.int64(445712), np.int64(445713), np.int64(445714), np.int64(445715), np.int64(445716), np.int64(445717), np.int64(445718), np.int64(445719), np.int64(445720), np.int64(445721), np.int64(445722), np.int64(445723), np.int64(445724), np.int64(445725), np.int64(445726), np.int64(445727), np.int64(445728), np.int64(445729), np.int64(445730), np.int64(445731), np.int64(445732), np.int64(445733), np.int64(445734), np.int64(445735), np.int64(445736), np.int64(445737), np.int64(445738), np.int64(445739), np.int64(445740), np.int64(445741), np.int64(445742), np.int64(445743), np.int64(445744), np.int64(445745), np.int64(445746), np.int64(445747), np.int64(445748), np.int64(445749), np.int64(445750), np.int64(445751), np.int64(445752), np.int64(445753), np.int64(445754), np.int64(163126), np.int64(163157), np.int64(445782), np.int64(445783), np.int64(445785), np.int64(445789), np.int64(445790), np.int64(445791), np.int64(445792), np.int64(445793), np.int64(445794), np.int64(146796), np.int64(443793), np.int64(443794), np.int64(443806), np.int64(163114), np.int64(163264), np.int64(445920), np.int64(445922), np.int64(445923), np.int64(445924), np.int64(445925), np.int64(445926), np.int64(445927), np.int64(445928), np.int64(445929), np.int64(445930), np.int64(445931), np.int64(445932), np.int64(445933), np.int64(445935), np.int64(445937), np.int64(445939), np.int64(445942), np.int64(445943), np.int64(445944), np.int64(445945), np.int64(445946), np.int64(445947), np.int64(445948), np.int64(445949), np.int64(445950), np.int64(445951), np.int64(445952), np.int64(445953), np.int64(445954), np.int64(445955), np.int64(445957), np.int64(445958), np.int64(445959), np.int64(445961), np.int64(445963), np.int64(445964), np.int64(445967), np.int64(443932), np.int64(445988), np.int64(157260), np.int64(157267), np.int64(446044), np.int64(446045), np.int64(446046), np.int64(446063), np.int64(446069), np.int64(32379), np.int64(446091), np.int64(446101), np.int64(446111), np.int64(444065), np.int64(446114), np.int64(446115), np.int64(446117), np.int64(446118), np.int64(446120), np.int64(446122), np.int64(446129), np.int64(446131), np.int64(446132), np.int64(446133), np.int64(446134), np.int64(446135), np.int64(446136), np.int64(446137), np.int64(446139), np.int64(444092), np.int64(446140), np.int64(446141), np.int64(446142), np.int64(446143), np.int64(446144), np.int64(446145), np.int64(446147), np.int64(446148), np.int64(446149), np.int64(446150), np.int64(446151), np.int64(446152), np.int64(446153), np.int64(446154), np.int64(446155), np.int64(446156), np.int64(446157), np.int64(446158), np.int64(446159), np.int64(446160), np.int64(446161), np.int64(446162), np.int64(446163), np.int64(446164), np.int64(446165), np.int64(446166), np.int64(446167), np.int64(446168), np.int64(446169), np.int64(446170), np.int64(446171), np.int64(446172), np.int64(446173), np.int64(446174), np.int64(437983), np.int64(446178), np.int64(446190), np.int64(446207), np.int64(446215), np.int64(446216), np.int64(446222), np.int64(446229), np.int64(446230), np.int64(446231), np.int64(446232), np.int64(446233), np.int64(446234), np.int64(446235), np.int64(446236), np.int64(446237), np.int64(446238), np.int64(444189), np.int64(446240), np.int64(446241), np.int64(446242), np.int64(446243), np.int64(446244), np.int64(446245), np.int64(446246), np.int64(446247), np.int64(446248), np.int64(446249), np.int64(446250), np.int64(446251), np.int64(446252), np.int64(446253), np.int64(446254), np.int64(446255), np.int64(446256), np.int64(444200), np.int64(446258), np.int64(446259), np.int64(446260), np.int64(446261), np.int64(446262), np.int64(446263), np.int64(446264), np.int64(446265), np.int64(446266), np.int64(446267), np.int64(446268), np.int64(446269), np.int64(446270), np.int64(446271), np.int64(446272), np.int64(446273), np.int64(446275), np.int64(446276), np.int64(446277), np.int64(446278), np.int64(446279), np.int64(446280), np.int64(446281), np.int64(446282), np.int64(446283), np.int64(446284), np.int64(446285), np.int64(446286), np.int64(446287), np.int64(446288), np.int64(446289), np.int64(446290), np.int64(446291), np.int64(446292), np.int64(446293), np.int64(446294), np.int64(446295), np.int64(446296), np.int64(446297), np.int64(446298), np.int64(446299), np.int64(446300), np.int64(446301), np.int64(438110), np.int64(446303), np.int64(446302), np.int64(446305), np.int64(446306), np.int64(446307), np.int64(446304), np.int64(446308), np.int64(446309), np.int64(446310), np.int64(446311), np.int64(446312), np.int64(446314), np.int64(446315), np.int64(446313), np.int64(446316), np.int64(446317), np.int64(446322), np.int64(446350), np.int64(446353), np.int64(446357), np.int64(446360), np.int64(446363), np.int64(446372), np.int64(446373), np.int64(446378), np.int64(446379), np.int64(446380), np.int64(446381), np.int64(446382), np.int64(446384), np.int64(446385), np.int64(446387), np.int64(446389), np.int64(446390), np.int64(446391), np.int64(446393), np.int64(446394), np.int64(446395), np.int64(446396), np.int64(446397), np.int64(446398), np.int64(446399), np.int64(446400), np.int64(446401), np.int64(444354), np.int64(446402), np.int64(446403), np.int64(446404), np.int64(446405), np.int64(446406), np.int64(446407), np.int64(446408), np.int64(446412), np.int64(446413), np.int64(446414), np.int64(446415), np.int64(446418), np.int64(446419), np.int64(446420), np.int64(446421), np.int64(446423), np.int64(446424), np.int64(446425), np.int64(446427), np.int64(446428), np.int64(446431), np.int64(446440), np.int64(446441), np.int64(446442), np.int64(446444), np.int64(446446), np.int64(446447), np.int64(446451), np.int64(446452), np.int64(446454), np.int64(446456), np.int64(446459), np.int64(446460), np.int64(446461), np.int64(446462), np.int64(446463)]
ad2 = ads_pool.loc[ads_pool.first_click >= '2025-08-22 00:00:00', 'ads_idx'].unique()


In [69]:
ads_pool.loc[ads_pool.first_click >= '2025-08-22 00:00:00', 'first_click'].unique()

array(['2025-08-22 04:25:52', '2025-08-22 08:07:56',
       '2025-08-24 10:30:04', '2025-08-24 14:12:27',
       '2025-08-22 22:39:29', '2025-08-23 22:18:48',
       '2025-08-22 20:09:09', '2025-08-22 01:43:45',
       '2025-08-23 22:41:51', '2025-08-24 00:34:39',
       '2025-08-23 23:10:47', '2025-08-23 07:15:45',
       '2025-08-22 05:26:17', '2025-08-22 18:20:25',
       '2025-08-24 23:02:19', '2025-08-23 19:01:01',
       '2025-08-25 04:54:32', '2025-08-22 09:10:10',
       '2025-08-22 20:11:47', '2025-08-25 00:22:05',
       '2025-08-25 07:22:32', '2025-08-23 22:13:17',
       '2025-08-23 12:15:19', '2025-08-25 00:50:35',
       '2025-08-25 00:40:09', '2025-08-24 00:36:22',
       '2025-08-24 00:36:13', '2025-08-25 00:45:42',
       '2025-08-22 00:59:43', '2025-08-24 00:35:24',
       '2025-08-24 00:14:22', '2025-08-22 00:59:57',
       '2025-08-22 06:20:50', '2025-08-22 10:25:19',
       '2025-08-23 12:17:39', '2025-08-24 12:08:00',
       '2025-08-24 12:06:14', '2025-08-24 03:4

In [68]:
list(ad2)

[np.int64(20889),
 np.int64(20914),
 np.int64(27829),
 np.int64(27879),
 np.int64(27898),
 np.int64(28686),
 np.int64(51557),
 np.int64(51576),
 np.int64(53595),
 np.int64(72057),
 np.int64(146796),
 np.int64(157260),
 np.int64(163114),
 np.int64(163126),
 np.int64(163157),
 np.int64(163264),
 np.int64(163267),
 np.int64(174805),
 np.int64(174808),
 np.int64(422797),
 np.int64(426935),
 np.int64(439558),
 np.int64(441107),
 np.int64(442885),
 np.int64(443175),
 np.int64(443793),
 np.int64(443794),
 np.int64(443802),
 np.int64(443806),
 np.int64(443932),
 np.int64(444065),
 np.int64(444092),
 np.int64(444189),
 np.int64(444200),
 np.int64(444354),
 np.int64(444479),
 np.int64(444499),
 np.int64(444577),
 np.int64(445029),
 np.int64(445031),
 np.int64(445033),
 np.int64(445044),
 np.int64(445072),
 np.int64(445073),
 np.int64(445074),
 np.int64(445075),
 np.int64(445087),
 np.int64(445097),
 np.int64(445098),
 np.int64(445104),
 np.int64(445108),
 np.int64(445112),
 np.int64(445124),
 np

In [62]:
ad3 = [446464, 446465, 446466, 446469, 446470, 446472, 446475, 446476,
       446478,  28686, 446480, 446482, 446483, 446484, 446486, 446489,
       446491, 446492, 446493, 446494, 446495, 446496, 446497, 446498,
       446499, 446500, 446501, 446502, 446503, 446504, 446505, 446506,
       446507, 446508, 446509, 446510, 446512, 446513, 446514, 446515,
       446517, 446518, 446519, 446521, 446525, 444479, 446544, 444499,
       446547, 446548, 446564, 446565, 446567, 446568, 446570, 446579,
       446580, 446582, 446583, 446585, 446586, 446587, 446588, 446589,
       446590, 446591, 446592, 446593, 446596, 446598, 446599, 446600,
       446605, 446607, 446611, 446613, 446622, 446623, 444577, 446630,
       446631, 446632, 446633, 446634, 446635, 446636, 446638, 446654,
       446668, 446671, 446672, 446673, 446674, 446675, 446676, 446678,
       446687, 446688, 446689, 446690, 446695, 446696, 446697, 446698,
       446699, 446700, 446701, 446702, 446703, 446704, 446705, 446706,
       446707, 446708, 446709, 446710, 446711, 446713, 446714, 446715,
       446716, 446717, 446718, 446719, 446720, 446721, 446722, 446723,
       446724, 446725, 446727, 446728, 446729, 446731, 446737, 446739,
       446740, 446741, 446742, 446743, 446744, 446755, 446757, 446759,
       446760, 446761, 446762, 446764, 446765, 446767, 446768, 446769,
       446771, 446773, 446774, 446778, 446782, 446786, 446788, 446789,
       446792, 446796, 446797, 446798, 446799, 446806, 446807, 446808,
       446809, 446810,  53595, 446811, 446812, 446819, 446820,  51557,
       446824, 446833, 446834, 446835, 446836, 446837, 446838, 446839,
        51576, 446840, 446841,  72057,  20889,  20914, 444933, 445029,
       445031, 445033, 445037, 445044, 445072, 445073, 445074, 445075,
       445087, 445097, 445098, 445108, 445112, 445124, 445131, 174805,
       174808, 445202, 445203, 441107, 394085, 445311, 445314, 414595,
       439558, 439192, 445359, 445360, 445361, 445450, 445451, 445453,
       445454, 445459, 445471, 445472, 445474, 445481, 445482, 445483,
       445484, 445485, 445486, 445487, 445488, 445489, 445491, 445492,
       445493, 445494, 445495, 445496, 445497, 445498, 445499, 445500,
       445501, 445502, 445503, 445504, 445507, 445508, 445509, 445510,
       445511, 445512, 445513, 445557, 445558, 445561, 445566, 445568,
       445577,  27829, 445637, 445657,  27879,  27898, 445701, 445702,
       445703, 445704, 445705, 445706, 445707, 445708, 445709, 445710,
       445711, 445712, 445713, 445714, 445715, 445716, 445717, 445718,
       445719, 445720, 445721, 445722, 445723, 445724, 445725, 445726,
       445727, 445728, 445729, 445730, 445731, 445732, 445733, 445734,
       445735, 445736, 445737, 445738, 445739, 445740, 445741, 445742,
       445743, 445744, 445745, 445746, 445747, 445748, 445749, 445750,
       445751, 445752, 445753, 445754, 163126, 163157, 445782, 445783,
       445785, 445789, 445790, 445791, 445792, 445793, 445794, 146796,
       443793, 443794, 443806, 163114, 163264, 445920, 445922, 445923,
       445924, 445925, 445926, 445927, 445928, 445929, 445930, 445931,
       445932, 445933, 445935, 445937, 445939, 445942, 445943, 445944,
       445945, 445946, 445947, 445948, 445949, 445950, 445951, 445952,
       445953, 445954, 445955, 445957, 445958, 445959, 445961, 445963,
       445964, 445967, 443932, 445988, 157260, 157267, 446044, 446045,
       446046, 446063, 446069,  32379, 446091, 446101, 446111, 444065,
       446114, 446115, 446117, 446118, 446120, 446122, 446129, 446131,
       446132, 446133, 446134, 446135, 446136, 446137, 446139, 444092,
       446140, 446141, 446142, 446143, 446144, 446145, 446147, 446148,
       446149, 446150, 446151, 446152, 446153, 446154, 446155, 446156,
       446157, 446158, 446159, 446160, 446161, 446162, 446163, 446164,
       446165, 446166, 446167, 446168, 446169, 446170, 446171, 446172,
       446173, 446174, 437983, 446178, 446190, 446207, 446215, 446216,
       446222, 446229, 446230, 446231, 446232, 446233, 446234, 446235,
       446236, 446237, 446238, 444189, 446240, 446241, 446242, 446243,
       446244, 446245, 446246, 446247, 446248, 446249, 446250, 446251,
       446252, 446253, 446254, 446255, 446256, 444200, 446258, 446259,
       446260, 446261, 446262, 446263, 446264, 446265, 446266, 446267,
       446268, 446269, 446270, 446271, 446272, 446273, 446275, 446276,
       446277, 446278, 446279, 446280, 446281, 446282, 446283, 446284,
       446285, 446286, 446287, 446288, 446289, 446290, 446291, 446292,
       446293, 446294, 446295, 446296, 446297, 446298, 446299, 446300,
       446301, 438110, 446303, 446302, 446305, 446306, 446307, 446304,
       446308, 446309, 446310, 446311, 446312, 446314, 446315, 446313,
       446316, 446317, 446322, 446350, 446353, 446357, 446360, 446363,
       446372, 446373, 446378, 446379, 446380, 446381, 446382, 446384,
       446385, 446387, 446389, 446390, 446391, 446393, 446394, 446395,
       446396, 446397, 446398, 446399, 446400, 446401, 444354, 446402,
       446403, 446404, 446405, 446406, 446407, 446408, 446412, 446413,
       446414, 446415, 446418, 446419, 446420, 446421, 446423, 446424,
       446425, 446427, 446428, 446431, 446440, 446441, 446442, 446444,
       446446, 446447, 446451, 446452, 446454, 446456, 446459, 446460,
       446461, 446462, 446463]

In [60]:
print(len(ad1),  len(list(ad2)))

603 367


In [66]:
set(ad2).issubset(ad3)  

False

In [74]:
list(set(ad2) - set(ad3)) # 84개

[np.int64(442885),
 np.int64(445971),
 np.int64(443802),
 np.int64(446516),
 np.int64(446520),
 np.int64(446009),
 np.int64(446546),
 np.int64(446052),
 np.int64(446064),
 np.int64(446077),
 np.int64(446109),
 np.int64(445104),
 np.int64(446138),
 np.int64(446679),
 np.int64(446680),
 np.int64(446681),
 np.int64(446682),
 np.int64(446686),
 np.int64(446191),
 np.int64(446198),
 np.int64(446208),
 np.int64(446730),
 np.int64(446219),
 np.int64(446228),
 np.int64(446745),
 np.int64(443175),
 np.int64(446813),
 np.int64(422797),
 np.int64(446826),
 np.int64(446827),
 np.int64(446828),
 np.int64(446320),
 np.int64(445300),
 np.int64(446850),
 np.int64(446851),
 np.int64(446852),
 np.int64(446853),
 np.int64(446854),
 np.int64(446855),
 np.int64(446861),
 np.int64(446862),
 np.int64(446863),
 np.int64(446864),
 np.int64(446865),
 np.int64(446866),
 np.int64(446867),
 np.int64(446868),
 np.int64(446869),
 np.int64(446870),
 np.int64(446871),
 np.int64(446872),
 np.int64(446873),
 np.int64(44

In [61]:
list(ad2)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

## 신규 광고 가상 시나리오

In [81]:
ads_pool.columns

Index(['ads_idx', 'media_count', 'user_count', 'total_clicks',
       'total_conversions', 'ads_category', 'domain', 'ads_os_type',
       'ads_order', 'ctit_mean', 'ctit_median', 'ads_rejoin_type',
       'contract_price', 'media_price', 'first_click', 'last_click',
       'ads_name', 'ads_sdate', 'expire', 'days_active',
       'daily_avg_conversions', 'cvr', 'margin', 'roi', 'total_net_return',
       'ads_level_detailed', 'daily_clicks', 'daily_users', 'scale_level',
       'performance_level', 'ads_level', 'media_score', 'conv_score',
       'clicks_score', 'stability_score', 'cvr_score', 'total_score',
       'ads_size', 'cluster', 'mda_idx_arr', 'M', 'A'],
      dtype='object')

In [94]:
help(pd.set_option)

Help on CallableDynamicDoc in module pandas._config.config:

<pandas._config.config.CallableDynamicDoc object>
    set_option(pat, value)
    
    Sets the value of the specified option.
    
    Available options:
    
    - compute.[use_bottleneck, use_numba, use_numexpr]
    - display.[chop_threshold, colheader_justify, date_dayfirst, date_yearfirst,
      encoding, expand_frame_repr, float_format]
    - display.html.[border, table_schema, use_mathjax]
    - display.[large_repr, max_categories, max_columns, max_colwidth, max_dir_items,
      max_info_columns, max_info_rows, max_rows, max_seq_items, memory_usage,
      min_rows, multi_sparse, notebook_repr_html, pprint_nest_depth, precision,
      show_dimensions]
    - display.unicode.[ambiguous_as_wide, east_asian_width]
    - display.[width]
    - future.[infer_string, no_silent_downcasting]
    - io.excel.ods.[reader, writer]
    - io.excel.xls.[reader]
    - io.excel.xlsb.[reader]
    - io.excel.xlsm.[reader, writer]
    - io.ex

In [95]:
pd.set_option('display.max_rows', 100)

In [96]:
ads_pool.value_counts(['domain', 'ads_category'])

domain   ads_category
커머스      4               928
식당/카페    4               886
생활       4               505
의료/건강    4               384
기타       4               289
게임       5               211
식음료      4               194
뷰티       4               175
커머스      11              171
운동/스포츠   4               136
뷰티       11              113
의료/건강    11               94
게임       6                83
미디어/컨텐츠  4                44
식음료      11               43
금융       10               39
         7                31
         4                30
게임       0                28
교육       4                25
생활       8                23
게임       2                22
운세       10               19
미디어/컨텐츠  8                18
금융       8                17
게임       4                15
운동/스포츠   11               11
생활       10               11
미디어/컨텐츠  10               10
금융       1                10
커머스      8                 9
비영리/공공   4                 8
커머스      10                7
식음료      5           

### 쇼핑

In [150]:
ads_list.loc[ads_list['ads_category'] == 11, 'ads_type'].value_counts()

ads_type
12    457
Name: count, dtype: int64

In [160]:
ads_list.loc[(ads_list['ads_category'] == 11)& (ads_list['ads_reward_price'] == 4000)]

,ads_idx,ads_code,aff_idx,adv_idx,ads_type,ads_category,ads_name,ads_icon_img,ads_summary,ads_save_way,ads_day_cap,ads_sdate,ads_edate,ads_age_min,ads_age_max,ads_os_type,ads_contract_price,ads_reward_price,ads_order,ads_rejoin_type,regdate,domain
446,150738,Eq0gpgn7li,1,1615,12,11,나틴다 리얼매직커버 선비비크림 50g,https://i.imgur.com/tlaR4zm.jpeg,제품을 구매/결제하시면 리워드를 즉시 지급해 드립니다.\n -본 이벤트는 제품을 구...,구매,False,2025-01-31 09:00:00,9999-12-31 00:00:00,0,100,7,6181,4000,1536307,REJOINABLE,2025-01-31 09:59:24 UTC,뷰티
448,150740,c5y1l8PR5b,1,1615,12,11,나틴다 알로에베라 딥 클렌징폼 130ml,https://i.imgur.com/w6efc6v.jpeg,제품을 구매/결제하시면 리워드를 즉시 지급해 드립니다.\n -본 이벤트는 제품을 구...,구매,False,2025-01-31 09:00:00,9999-12-31 00:00:00,0,100,7,6181,4000,1540795,REJOINABLE,2025-01-31 09:59:52 UTC,뷰티
482,157242,vRYuc0n012,1,930,12,11,2+1 블랙앵거스 양념 LA갈비1kg+500g,https://gi.esmplus.com/glodaon/%EC%95%84%EC%9D...,제품을 구매/결제하시면 리워드를 즉시 지급해 드립니다.\n -본 이벤트는 제품을 구...,구매,False,2025-02-17 11:00:00,9999-12-31 00:00:00,0,100,7,6181,4000,1623922,REJOINABLE,2025-02-17 11:10:10 UTC,커머스
691,254797,9a6g1IGOCV,1,1615,12,11,서울약사신협 락토온 WHY Y케어 30포,https://i.imgur.com/K44dqME.jpeg,제품을 구매/결제하시면 리워드를 즉시 지급해 드립니다.\n -본 이벤트는 제품을 구...,구매,False,2025-04-02 10:00:00,9999-12-31 00:00:00,0,100,7,6236,4000,1533090,REJOINABLE,2025-04-02 10:39:36 UTC,의료/건강
947,414611,ENfsA15TLX,1,1615,12,11,국물 닭한마리 볶음탕 1.4kg,https://i.imgur.com/iw1mWTm.jpeg,제품을 구매/결제하시면 리워드를 즉시 지급해 드립니다.\n -본 이벤트는 제품을 구...,구매,False,2025-07-04 11:00:00,9999-12-31 00:00:00,0,100,7,6272,4000,1631793,REJOINABLE,2025-07-04 11:00:42 UTC,식음료
3072,437723,H5sozSE4pz,1,1615,12,11,국물 닭한마리 볶음탕 1.4kg,https://i.imgur.com/7p0ZYn6.jpeg,제품을 구매/결제하시면 리워드를 즉시 지급해 드립니다.\n -본 이벤트는 제품을 구...,구매,False,2025-07-28 13:00:00,9999-12-31 00:00:00,0,100,7,6272,4000,1639409,REJOINABLE,2025-07-28 13:38:53 UTC,식음료


In [161]:
ads_pool.loc[(ads_pool['ads_category'] == 11)&(ads_pool['media_price'] >= 4000)].sort_values('media_price').head(100)

,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,ads_order,ctit_mean,ctit_median,ads_rejoin_type,contract_price,media_price,first_click,last_click,ads_name,ads_sdate,expire,days_active,daily_avg_conversions,cvr,margin,roi,total_net_return,ads_level_detailed,daily_clicks,daily_users,scale_level,performance_level,ads_level,media_score,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster,mda_idx_arr,M,A
416,150740,2,2,6,0,11,뷰티,7,1540795,NaN,NaN,REJOINABLE,6181,4000,2025-07-26 10:58:17,2025-08-19 21:38:57,나틴다 알로에베라 딥 클렌징폼 130ml,2025-01-31 09:00:00,0,25,0.000000,0.0,2181,0.5,0,LARGE,0.240000,0.080000,MEDIUM,LOW,MEDIUM_LOW,2,0,0,3,0,5,MEDIUM,0,NaN,0.0,1.0
450,157242,3,11,15,1,11,커머스,7,1623922,144.000000,144.0,REJOINABLE,6181,4000,2025-07-29 12:36:28,2025-08-25 04:41:22,2+1 블랙앵거스 양념 LA갈비1kg+500g,2025-02-17 11:00:00,0,27,0.037037,0.1,2181,0.5,2181,LARGE,0.555556,0.407407,MEDIUM,LOW,MEDIUM_LOW,2,0,0,3,0,5,MEDIUM,0,NaN,0.0,1.0
880,414611,4,4,6,0,11,식음료,7,1631793,NaN,NaN,REJOINABLE,6272,4000,2025-07-26 13:45:53,2025-08-22 10:17:32,국물 닭한마리 볶음탕 1.4kg,2025-07-04 11:00:00,0,27,0.000000,0.0,2272,0.6,0,LARGE,0.222222,0.148148,LARGE,LOW,LARGE_LOW,3,0,0,3,0,6,MEDIUM,0,NaN,0.0,1.0
1746,437723,3,6,6,0,11,식음료,7,1639409,NaN,NaN,REJOINABLE,6272,4000,2025-07-30 12:38:48,2025-08-22 10:23:41,국물 닭한마리 볶음탕 1.4kg,2025-07-28 13:00:00,0,23,0.000000,0.0,2272,0.6,0,LARGE,0.260870,0.260870,MEDIUM,LOW,MEDIUM_LOW,2,0,0,3,0,5,MEDIUM,0,NaN,0.0,1.0
656,254797,1,1,1,0,11,의료/건강,7,1533090,NaN,NaN,REJOINABLE,6236,4000,2025-07-29 06:02:14,2025-07-29 06:02:14,서울약사신협 락토온 WHY Y케어 30포,2025-04-02 10:00:00,0,1,0.000000,0.0,2236,0.6,0,LARGE,1.000000,1.000000,SMALL,LOW,SMALL_LOW,1,0,0,0,0,1,SMALL,0,NaN,0.0,1.0
414,150738,1,1,9,0,11,뷰티,7,1536307,NaN,NaN,REJOINABLE,6181,4000,2025-07-28 21:03:37,2025-08-17 00:10:03,나틴다 리얼매직커버 선비비크림 50g,2025-01-31 09:00:00,0,20,0.000000,0.0,2181,0.5,0,LARGE,0.450000,0.050000,SMALL,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0,NaN,0.0,1.0
604,174792,3,3,5,1,11,식음료,7,1598870,133.000000,133.0,REJOINABLE,9000,4060,2025-08-02 17:16:21,2025-08-16 09:08:20,달콤한 레몬어트 유기농 NFC 레몬즙 14포,2025-03-14 18:00:00,0,14,0.071429,0.2,4940,1.2,4940,LARGE,0.357143,0.214286,MEDIUM,LOW,MEDIUM_LOW,2,0,0,2,0,4,SMALL,0,NaN,0.0,1.0
181,28686,1,1,1,0,11,의료/건강,7,982320,NaN,NaN,REJOINABLE,5801,4060,2025-08-23 22:18:48,2025-08-23 22:18:48,[슈퍼푸드K] 그린바나나 정 600mg x 100정,2024-02-16 16:41:31,0,1,0.000000,0.0,1741,0.4,0,LARGE,1.000000,1.000000,SMALL,LOW,SMALL_LOW,1,0,0,0,0,1,SMALL,0,NaN,0.0,1.0
808,385711,2,1,6,0,11,뷰티,7,1369941,NaN,NaN,REJOINABLE,6363,4100,2025-07-30 09:16:58,2025-08-21 04:58:18,아론샵 원스텝 스트레이트 펌 매직약 150ml,2025-06-12 15:00:00,0,22,0.000000,0.0,2263,0.6,0,LARGE,0.272727,0.045455,MEDIUM,LOW,MEDIUM_LOW,2,0,0,3,0,5,MEDIUM,0,NaN,0.0,1.0
960,430432,3,3,3,0,11,커머스,7,2755400,NaN,NaN,NONE,5600,4200,2025-07-27 22:49:12,2025-07-31 12:58:33,락앤락 첫구매 이벤트,2025-07-15 13:00:00,0,4,0.000000,0.0,1400,0.3,0,LARGE,0.750000,0.750000,MEDIUM,LOW,MEDIUM_LOW,2,0,0,1,0,3,SMALL,0,NaN,0.0,1.0


### 게임


 OS 타입 (1:AppStore, 2:GooglePlay, 3:원스토어, 7:Web, 8:갤럭시 스토어) 


In [179]:
ads_list.loc[(ads_list['ads_os_type'] == 1) & (ads_pool['domain']=='게임')].value_counts('ads_rejoin_type') #'domain'

ads_rejoin_type
NONE    38
Name: count, dtype: int64

contract_price	media_price
mean	3276.888889	2457.407407
median	2400.000000	1800.000000

In [199]:
game_os_1 = ads_pool.loc[(ads_pool['domain']=='게임') & (ads_pool['ads_os_type'] == 1) & (ads_pool['mda_idx_arr'].isna()) & (ads_pool['ads_size'] == 'SMALL'), 'ads_idx'] # .agg(['mean', 'median']) #.value_counts('ads_category') # 

In [205]:
ads_pool.loc[(ads_pool['domain']=='게임') & (ads_pool['ads_os_type'] == 2) & (ads_pool['mda_idx_arr'].isna()) & (ads_pool['ads_size'] == 'SMALL')]

,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,ads_order,ctit_mean,ctit_median,ads_rejoin_type,contract_price,media_price,first_click,last_click,ads_name,ads_sdate,expire,days_active,daily_avg_conversions,cvr,margin,roi,total_net_return,ads_level_detailed,daily_clicks,daily_users,scale_level,performance_level,ads_level,media_score,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster,mda_idx_arr,M,A
49,17234,1,16,39,8,0,게임,2,1823000,137977.125000,120622.0,NONE,0,399,2025-07-26 15:26:49,2025-08-24 11:55:17,IDLE ANGELS : 여신전쟁,2022-05-20 17:00:00,0,29,0.275862,0.2,-399,-1.0,-3192,SMALL,1.344828,0.551724,SMALL,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0,NaN,0.0,1.0
50,17245,1,9,18,4,0,게임,2,1823800,214268.750000,176517.0,NONE,0,175,2025-07-31 20:50:16,2025-08-25 07:55:35,전국구 : 권력의 법칙,2022-05-20 17:00:00,0,25,0.160000,0.2,-175,-1.0,-700,MICRO,0.720000,0.360000,SMALL,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0,NaN,0.0,1.0
59,19247,1,5,17,4,0,게임,2,1945800,173513.250000,158367.0,NONE,0,354,2025-08-07 07:44:20,2025-08-21 09:23:29,천하제일검,2022-09-07 16:00:00,0,15,0.266667,0.2,-354,-1.0,-1416,MICRO,1.133333,0.333333,SMALL,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0,NaN,0.0,1.0
62,19953,1,12,39,8,0,게임,2,1968900,208159.375000,134915.0,NONE,0,350,2025-07-28 12:58:48,2025-08-24 18:22:52,조선좀비전 : 방치형 RPG,2022-10-24 18:00:00,0,28,0.285714,0.2,-350,-1.0,-2800,SMALL,1.392857,0.428571,SMALL,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0,NaN,0.0,1.0
63,20062,1,12,24,3,0,게임,2,1971800,89679.666667,19074.0,NONE,0,179,2025-07-28 20:04:19,2025-08-25 07:55:47,정령사 키우기,2022-10-28 10:00:00,0,28,0.107143,0.1,-179,-1.0,-537,MICRO,0.857143,0.428571,SMALL,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0,NaN,0.0,1.0
64,20168,1,9,20,5,6,게임,2,1973100,33121.400000,31552.0,NONE,0,124,2025-07-27 10:06:06,2025-08-20 15:10:13,계급장 키우기 대폭발,2022-11-03 10:00:00,0,25,0.200000,0.2,-124,-1.0,-620,MICRO,0.800000,0.360000,SMALL,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0,NaN,0.0,1.0
66,20195,1,13,29,5,0,게임,2,1975600,179513.200000,128909.0,NONE,0,350,2025-07-27 23:57:09,2025-08-24 12:15:34,소녀전쟁: 환상의 천하통일전,2022-11-04 14:00:00,0,28,0.178571,0.2,-350,-1.0,-1750,MICRO,1.035714,0.464286,SMALL,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0,NaN,0.0,1.0
70,20328,1,10,20,6,6,게임,2,1980800,365971.666667,203451.5,NONE,0,205,2025-07-26 05:06:33,2025-08-23 12:00:30,테이밍 마스터,2022-11-15 15:00:00,0,29,0.206897,0.3,-205,-1.0,-1230,MICRO,0.689655,0.344828,SMALL,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0,NaN,0.0,1.0
75,20705,1,5,10,5,6,게임,2,2004300,208160.600000,212065.0,NONE,0,105,2025-07-30 08:24:11,2025-08-16 01:12:50,트리플 판타지 - 카드 마스터,2022-12-30 14:00:00,0,17,0.294118,0.5,-105,-1.0,-525,MICRO,0.588235,0.294118,SMALL,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0,NaN,0.0,1.0
76,20726,1,8,19,4,6,게임,2,2009200,89300.750000,90026.5,NONE,0,305,2025-07-30 07:48:14,2025-08-20 01:26:09,우주소녀 키우기:방치형RPG,2023-01-11 16:00:00,0,21,0.190476,0.2,-305,-1.0,-1220,MICRO,0.904762,0.380952,SMALL,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0,NaN,0.0,1.0


In [209]:
ads_list[ads_list['ads_category'] == 5].value_counts('ads_type')
# ads_rejoin_type

ads_rejoin_type
NONE    232
Name: count, dtype: int64

In [203]:
ads_list.loc[ads_list['ads_idx'].isin(game_os_1)] # , ['ads_contract_price', 'ads_reward_price'] .agg(['mean', 'median'])

,ads_idx,ads_code,aff_idx,adv_idx,ads_type,ads_category,ads_name,ads_icon_img,ads_summary,ads_save_way,ads_day_cap,ads_sdate,ads_edate,ads_age_min,ads_age_max,ads_os_type,ads_contract_price,ads_reward_price,ads_order,ads_rejoin_type,regdate,domain
155,27175,T0kOXZ2cNM,6,10,3,2,판타지타운(레벨27달성),https://nextapps-nas.aws.appang.kr/icon/202108...,[레벨27달성/최초설치자만] 간만에 대작 SNG! 판타지 마을을 경영하세요.\n\n...,최초 설치 후 레벨 27 달성,False,2023-12-06 11:00:00,2029-11-29 00:00:00,0,100,1,4000,3000,2438800,NONE,2023-12-06 11:10:39 UTC,게임
526,163114,t3YT8O65mv,76,1355,3,5,브레인 테스트 (Brain Test): 까다로운 퍼즐(Level 400 도달),https://cf.smaad.net/435490670/501475862.png,두뇌 테스트는 중독성이 있고 교묘한 무료 퍼즐 게임입니다. 재미있고 교묘한 퍼즐이 ...,Level 400 도달,False,2025-02-27 15:00:00,2035-01-01 00:00:00,0,100,1,2640,1980,2443900,NONE,2025-02-26 16:18:51 UTC,게임
528,163118,ZKv6LQn7nX,76,1355,3,5,퍼즐 오브 Z (지휘센터 Lv.17 도달),https://cf.smaad.net/463968598/991270900.png,글로벌 다운로드 수 1500만 돌파! 퍼즐 SLG 대작 ‘퍼즐 오브 Z’ 한국 출시...,지휘센터 Lv.17 도달,False,2025-02-27 16:00:00,2031-01-04 00:00:00,0,100,1,14400,10800,2475500,NONE,2025-02-26 16:20:55 UTC,게임
532,163126,t94vS7XC4t,76,1355,3,5,버스 열풍 - 교통 체증 퍼즐,https://cf.smaad.net/898310852/410906128.png,스릴 넘치는 새로운 교통 정체 퍼즐 게임인 Bus Craze에서 도로를 장악하고 혼...,레벨 500 클리어,False,2025-02-27 16:00:00,2035-01-01 00:00:00,0,100,1,4800,3600,2457300,NONE,2025-02-26 16:29:10 UTC,게임
536,163134,ChRRGWmeND,76,1355,3,5,개미 군단,https://cf.smaad.net/985416175/926948233.png,빙하 속에서 개미굴을 이끌고 생존공간을 쟁탈합니다.\n\n\n▼승인 조건\n신규 앱...,여왕개미 Lv.12 클리어,False,2025-02-27 16:00:00,2035-01-01 00:00:00,0,100,1,6400,4800,2460600,NONE,2025-02-26 16:33:24 UTC,게임
542,163144,cgd7EVC22Q,76,1355,3,5,Sea Block 1010(레벨10 도달),https://cf.smaad.net/249333457/965127743.png,Sea Block 1010이라는 전통적인 퍼즐 블록 게임의 독특한 변형은 아름다운 ...,레벨10 도달,False,2025-02-27 16:00:00,2035-01-01 00:00:00,0,100,1,1120,840,2461400,NONE,2025-02-26 16:37:13 UTC,게임
549,163157,sUaoZSXZir,76,1355,3,5,스매시 파티(Smash Party),https://cf.smaad.net/947375214/847767238.png,"스매시 파티 세계에 오신 것을 환영합니다! 경찰대 여러분, 정체불명의 갱단에게 점거...",노말모드에서 월드8 클리어,False,2025-02-27 16:00:00,2035-01-24 00:00:00,0,100,1,8000,6000,2457400,NONE,2025-02-26 16:47:25 UTC,게임
550,163160,6u7u33uqkp,76,1355,3,5,Find Out: 숨은 물건 찾기,https://cf.smaad.net/419366192/111818838.png,관찰력을 발휘해 그림 속에 숨어 있는 물건들을 모두 찾아내면 스테이지를 클리어할 수...,별450개 획득,False,2025-02-27 16:00:00,2035-01-01 00:00:00,0,100,1,2800,2100,2438700,NONE,2025-02-26 16:50:31 UTC,게임
555,163169,5DG6ZLJllc,76,1355,3,5,Wood Guy,https://cf.smaad.net/409128194/176627845.png,나무 인형으로 물리 법칙을 무시하고 수공예품의 세계로 여행을 떠나세요!\n\n▼승인...,레벨300도달,False,2025-02-27 16:00:00,2035-01-01 00:00:00,0,100,1,1920,1440,2453000,NONE,2025-02-26 16:56:10 UTC,게임
556,163170,SMYVj7H10s,76,1355,3,5,앨리스 어드벤처 - Merge Game,https://cf.smaad.net/135388527/142386237.png,에일리가 기이한 사건과 새로운 도전을 만날 때 어떻게 대처할까요?\n\n▼승인 조건...,레벨10 도달,False,2025-02-27 16:00:00,2035-01-01 00:00:00,0,100,1,3600,2700,2449000,NONE,2025-02-26 16:58:49 UTC,게임


In [105]:
ads_pool.loc[ads_pool['domain']=='게임'].value_counts(['ads_category', 'ads_os_type'])

ads_category  ads_os_type
5             2              155
6             2               79
5             1               52
0             2               28
2             2               16
4             7               15
2             1                5
5             7                4
1             7                3
6             3                3
8             2                2
2             3                1
1             2                1
              3                1
6             7                1
8             7                1
10            7                1
Name: count, dtype: int64

#### 앱 설치

In [208]:
ads_list.loc[ads_list['ads_category'] == 6, 'ads_type'].value_counts()

ads_type
9    86
Name: count, dtype: int64

In [207]:
ads_pool.loc[ads_pool['ads_category'] == 6]

,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,ads_order,ctit_mean,ctit_median,ads_rejoin_type,contract_price,media_price,first_click,last_click,ads_name,ads_sdate,expire,days_active,daily_avg_conversions,cvr,margin,roi,total_net_return,ads_level_detailed,daily_clicks,daily_users,scale_level,performance_level,ads_level,media_score,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster,mda_idx_arr,M,A
25,11123,1,12,13,8,6,게임,2,1429100,36649.250000,19320.5,NONE,0,300,2025-07-28 17:40:19,2025-08-21 00:29:48,프린세스테일,2021-05-10 14:00:00,0,24,0.333333,0.6,-300,-1.0,-2400,MICRO,0.541667,0.500000,SMALL,LOW,SMALL_LOW,1,0,0,3,1,5,MEDIUM,0,NaN,0.0,1.0
38,16485,1,43,99,11,6,게임,2,1773300,158545.272727,64260.0,NONE,0,475,2025-07-26 00:49:55,2025-08-24 12:56:15,브롤스타즈,2022-04-18 19:00:00,0,30,0.366667,0.1,-475,-1.0,-5225,MEDIUM,3.300000,1.433333,SMALL,LOW,SMALL_LOW,1,0,1,3,0,5,MEDIUM,0,NaN,0.0,1.0
61,19853,1,35,117,11,6,게임,2,1966400,60621.909091,60271.0,NONE,0,474,2025-07-26 12:30:17,2025-08-24 12:25:31,계급장 키우기,2022-10-14 11:00:00,0,29,0.379310,0.1,-474,-1.0,-5214,MEDIUM,4.034483,1.206897,SMALL,LOW,SMALL_LOW,1,0,1,3,0,5,MEDIUM,0,NaN,0.0,1.0
64,20168,1,9,20,5,6,게임,2,1973100,33121.400000,31552.0,NONE,0,124,2025-07-27 10:06:06,2025-08-20 15:10:13,계급장 키우기 대폭발,2022-11-03 10:00:00,0,25,0.200000,0.2,-124,-1.0,-620,MICRO,0.800000,0.360000,SMALL,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0,NaN,0.0,1.0
70,20328,1,10,20,6,6,게임,2,1980800,365971.666667,203451.5,NONE,0,205,2025-07-26 05:06:33,2025-08-23 12:00:30,테이밍 마스터,2022-11-15 15:00:00,0,29,0.206897,0.3,-205,-1.0,-1230,MICRO,0.689655,0.344828,SMALL,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0,NaN,0.0,1.0
73,20583,1,42,81,1,6,게임,2,1995900,362308.000000,362308.0,NONE,0,499,2025-07-26 00:48:21,2025-08-24 12:01:51,Doge Rescue: Draw to save,2022-12-09 14:00:00,0,30,0.033333,0.0,-499,-1.0,-499,MEDIUM,2.700000,1.400000,SMALL,LOW,SMALL_LOW,1,0,1,3,0,5,MEDIUM,0,NaN,0.0,1.0
75,20705,1,5,10,5,6,게임,2,2004300,208160.600000,212065.0,NONE,0,105,2025-07-30 08:24:11,2025-08-16 01:12:50,트리플 판타지 - 카드 마스터,2022-12-30 14:00:00,0,17,0.294118,0.5,-105,-1.0,-525,MICRO,0.588235,0.294118,SMALL,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0,NaN,0.0,1.0
76,20726,1,8,19,4,6,게임,2,2009200,89300.750000,90026.5,NONE,0,305,2025-07-30 07:48:14,2025-08-20 01:26:09,우주소녀 키우기:방치형RPG,2023-01-11 16:00:00,0,21,0.190476,0.2,-305,-1.0,-1220,MICRO,0.904762,0.380952,SMALL,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0,NaN,0.0,1.0
77,20737,1,10,21,5,6,게임,2,2006600,86414.800000,39735.0,NONE,0,329,2025-07-28 16:04:29,2025-08-21 16:30:22,열혈 방랑객,2023-01-07 12:00:00,0,25,0.200000,0.2,-329,-1.0,-1645,MICRO,0.840000,0.400000,SMALL,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0,NaN,0.0,1.0
79,20801,1,8,23,5,6,게임,2,2013800,145502.400000,140874.0,NONE,0,54,2025-08-04 09:51:04,2025-08-21 01:46:36,전삼국,2023-01-19 14:00:00,0,17,0.294118,0.2,-54,-1.0,-270,MICRO,1.352941,0.470588,SMALL,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0,NaN,0.0,1.0


### category 4 - 퀴즈형

In [100]:
ads_pool.loc[ads_pool['ads_category'] == 4, 'ads_os_type'].value_counts()

ads_os_type
7    3621
Name: count, dtype: int64

In [149]:
ads_list.loc[(ads_list['ads_category'] == 4) & (ads_list.ads_rejoin_type == 'ADS_CODE_DAILY_UPDATE') ] #.value_counts()

,ads_idx,ads_code,aff_idx,adv_idx,ads_type,ads_category,ads_name,ads_icon_img,ads_summary,ads_save_way,ads_day_cap,ads_sdate,ads_edate,ads_age_min,ads_age_max,ads_os_type,ads_contract_price,ads_reward_price,ads_order,ads_rejoin_type,regdate,domain
695,254824,NzrrYSZQr7,1,1512,3,4,[개발실 테스트] 쿠팡 출석체크,https://imgur.com/bp7QYuc.png,쿠팡에서 상품 구입,참여,False,2025-04-02 11:00:00,2026-01-01 00:00:00,0,100,7,10,5,2487600,ADS_CODE_DAILY_UPDATE,2025-04-02 11:47:41 UTC,커머스
810,373073,NMxunYzTYv,82,1596,3,4,[정답입력] 춘천맛집,https://cdn.quizclick.io/quiz/public/campaign/...,[참여 방법] \n1. 광고 참여 버튼 터치하여 이벤트 페이지로 이동 \n2. 이벤...,네이버 플레이스 지도 거리 입력하기,False,2025-05-30 19:32:15,9999-12-31 23:59:59,0,100,7,18,12,1369139,ADS_CODE_DAILY_UPDATE,2025-05-30 19:32:15 UTC,식당/카페
846,384191,7SxQmIZKmB,82,1596,3,4,[정답입력] 노량진맛집,https://cdn.quizclick.io/quiz/public/campaign/...,[참여 방법] \n1. 광고 참여 버튼 터치하여 이벤트 페이지로 이동 \n2. 이벤...,네이버 플레이스 지도 거리 입력하기,False,2025-06-10 21:27:09,9999-12-31 23:59:59,0,100,7,18,12,1369271,ADS_CODE_DAILY_UPDATE,2025-06-10 21:27:09 UTC,식당/카페
865,386998,DAOkyvpDNM,82,1596,3,4,[정답입력]광명맛집,https://cdn.quizclick.io/quiz/public/campaign/...,[참여 방법] \n1. 광고 참여 버튼 터치하여 이벤트 페이지로 이동 \n2. 이벤...,네이버 플레이스 지도 거리 입력하기,False,2025-06-13 19:22:30,9999-12-31 23:59:59,0,100,7,18,12,1369379,ADS_CODE_DAILY_UPDATE,2025-06-13 19:22:30 UTC,식당/카페
866,387001,zfhtQVBdDA,82,1596,3,4,[정답입력] 범계맛집,https://cdn.quizclick.io/quiz/public/campaign/...,[참여 방법] \n1. 광고 참여 버튼 터치하여 이벤트 페이지로 이동 \n2. 이벤...,네이버 플레이스 지도 거리 입력하기,False,2025-06-13 19:42:18,9999-12-31 23:59:59,0,100,7,18,12,1369379,ADS_CODE_DAILY_UPDATE,2025-06-13 19:42:18 UTC,식당/카페
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9510,446897,n2dnMyhfqM,86,1655,3,4,완벽한일상 180210,https://cashplan-r2.uk/quiz-mission.png,[참여방법]\n1. 본문에서 퀴즈와 상품/장소 확인\n1-1. 퀴즈: 판매자 정보에...,네이버스마트스토어 퀴즈,False,2025-08-25 09:37:26,9999-12-31 23:59:59,0,100,7,18,12,1369916,ADS_CODE_DAILY_UPDATE,2025-08-25 09:37:26 UTC,커머스
9511,446898,LHtyLlTU9v,86,1655,3,4,청색롤테이너 고리형 600x900x1500 180211,https://cashplan-r2.uk/quiz-mission.png,[참여방법]\n1. 본문에서 퀴즈와 상품/장소 확인\n1-1. 퀴즈: 8번째 태그 ...,네이버스마트스토어 퀴즈,False,2025-08-25 09:37:27,9999-12-31 23:59:59,0,100,7,18,12,1369916,ADS_CODE_DAILY_UPDATE,2025-08-25 09:37:26 UTC,커머스
9512,446899,U80Aed1tRq,86,1655,3,4,원스토몰 180212,https://cashplan-r2.uk/quiz-mission.png,[참여방법]\n1. 본문에서 퀴즈와 상품/장소 확인\n1-1. 퀴즈: 4번째 태그 ...,네이버스마트스토어 퀴즈,False,2025-08-25 09:37:27,9999-12-31 23:59:59,0,100,7,18,12,1369916,ADS_CODE_DAILY_UPDATE,2025-08-25 09:37:27 UTC,커머스
9513,446900,dfX4EjOsTE,86,1655,3,4,서빙카트 식당 업소용 주방 서빙카 다용도 서빙카트 소형 2단 기본고무바퀴 180213,https://cashplan-r2.uk/quiz-mission.png,[참여방법]\n1. 본문에서 퀴즈와 상품/장소 확인\n1-1. 퀴즈: 7번째 태그 ...,네이버스마트스토어 퀴즈,False,2025-08-25 09:42:08,9999-12-31 23:59:59,0,100,7,18,12,1369915,ADS_CODE_DAILY_UPDATE,2025-08-25 09:42:08 UTC,식당/카페


In [124]:
ads_list[ads_list.ads_idx.isin([399394, 435000, 436820, 437600])]

,ads_idx,ads_code,aff_idx,adv_idx,ads_type,ads_category,ads_name,ads_icon_img,ads_summary,ads_save_way,ads_day_cap,ads_sdate,ads_edate,ads_age_min,ads_age_max,ads_os_type,ads_contract_price,ads_reward_price,ads_order,ads_rejoin_type,regdate,domain
892,399394,4M8fgycgzq,85,1631,3,4,축석장어랜드 포천 광릉수목원점,https://i.imgur.com/XdcGFx7.jpeg,"1. 가이드라인 대로 진행하지 않으시면, 매크로로 인식하여 추후 문제가 발생할 수...",정답입력,False,2025-06-24 00:15:11,9999-12-31 23:59:59,0,100,7,16,10,1369379,ADS_CODE_DAILY_UPDATE,2025-06-24 00:15:11 UTC,식당/카페
1399,435000,HVPZ5cO9lS,85,1631,3,4,위너스장기주차장실내예약 인천공항로점,https://i.imgur.com/XdcGFx7.jpeg,1.'네이버' 를 클릭해 주세요 \n\n2.복사한 키워드를 네이버 검색창에 붙여넣기...,정답입력,False,2025-07-22 14:02:22,9999-12-31 23:59:59,0,100,7,16,10,1369610,ADS_CODE_DAILY_UPDATE,2025-07-22 14:02:22 UTC,생활
2357,436820,awKbkZhLGW,85,1631,3,4,고잔갈비 안산본점,https://i.imgur.com/XdcGFx7.jpeg,1.'네이버' 를 클릭해 주세요 \n\n2.복사한 키워드를 네이버 검색창에 붙여넣기...,정답입력,False,2025-07-26 04:53:10,9999-12-31 23:59:59,0,100,7,16,10,1369610,ADS_CODE_DAILY_UPDATE,2025-07-26 04:53:09 UTC,식당/카페
2985,437600,UwVfO65vGB,85,1631,3,4,따봉시래기 신현점,https://i.imgur.com/XdcGFx7.jpeg,"1. 가이드라인 대로 진행하지 않으시면, 매크로로 인식하여 추후 문제가 발생할 수...",정답입력,False,2025-07-27 21:42:02,9999-12-31 23:59:59,0,100,7,16,10,1369610,ADS_CODE_DAILY_UPDATE,2025-07-27 21:42:02 UTC,식당/카페


In [134]:
ads_pool.loc[(ads_pool['ads_category'] == 4) & (ads_pool.ads_rejoin_type == 'ADS_CODE_DAILY_UPDATE') & (ads_pool['mda_idx_arr'].isna())].value_counts('domain')

domain
커머스       68
식당/카페     23
생활        12
의료/건강     11
뷰티        10
식음료        7
운동/스포츠     6
금융         1
Name: count, dtype: int64

In [137]:
print(68/138, 23/138)

0.4927536231884058 0.16666666666666666


In [144]:
ads_pool.loc[(ads_pool['ads_category'] == 4) & (ads_pool.ads_rejoin_type == 'ADS_CODE_DAILY_UPDATE') & (ads_pool['mda_idx_arr'].isna()) &(ads_pool['ads_size'] == 'LARGE')] #.value_counts('ads_size')

,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,ads_order,ctit_mean,ctit_median,ads_rejoin_type,contract_price,media_price,first_click,last_click,ads_name,ads_sdate,expire,days_active,daily_avg_conversions,cvr,margin,roi,total_net_return,ads_level_detailed,daily_clicks,daily_users,scale_level,performance_level,ads_level,media_score,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster,mda_idx_arr,M,A
4562,446673,3,205,230,203,4,생활,7,1369916,42.408867,33.0,ADS_CODE_DAILY_UPDATE,20,16,2025-08-24 00:30:29,2025-08-24 00:52:42,유성전자담배 타임월드점 (플레이스 보고 퀴즈 맞추기),2025-08-24 00:07:49,0,1,203.0,0.9,4,0.2,812,MEDIUM,230.0,205.0,MEGA,HIGH,MEGA_HIGH,2,4,3,0,2,11,LARGE,0,NaN,0.0,0.0
4563,446674,3,203,230,203,4,생활,7,1369916,34.019704,29.0,ADS_CODE_DAILY_UPDATE,20,16,2025-08-24 00:30:38,2025-08-24 00:54:04,대전 유성전자담배 도안점 (플레이스 보고 퀴즈 맞추기),2025-08-24 00:07:50,0,1,203.0,0.9,4,0.2,812,MEDIUM,230.0,203.0,MEGA,HIGH,MEGA_HIGH,2,4,3,0,2,11,LARGE,0,NaN,0.0,0.0
4564,446675,3,204,233,204,4,생활,7,1369916,33.215686,29.0,ADS_CODE_DAILY_UPDATE,20,16,2025-08-24 00:30:44,2025-08-24 00:55:09,유성전자담배 본점 (플레이스 보고 퀴즈 맞추기),2025-08-24 00:07:51,0,1,204.0,0.9,4,0.2,816,MEDIUM,233.0,204.0,MEGA,HIGH,MEGA_HIGH,2,4,3,0,2,11,LARGE,0,NaN,0.0,0.0
4565,446676,3,109,121,109,4,생활,7,1369916,30.880734,27.0,ADS_CODE_DAILY_UPDATE,20,16,2025-08-24 00:30:33,2025-08-24 00:37:32,유성전자담배 청주강서점 (플레이스 보고 퀴즈 맞추기),2025-08-24 00:07:51,0,1,109.0,0.9,4,0.2,436,MEDIUM,121.0,109.0,MEGA,HIGH,MEGA_HIGH,2,3,3,0,2,10,LARGE,0,NaN,0.0,0.0
4572,446687,2,33,38,30,4,커머스,7,1369916,65.466667,45.5,ADS_CODE_DAILY_UPDATE,18,14,2025-08-24 04:10:59,2025-08-25 08:28:10,1+1 리빙채널 남녀공용 발등낮은 욕실 화장실 베란다 논슬립 욕실화 슬리퍼 179698,2025-08-24 00:09:10,0,2,15.0,0.8,4,0.3,120,SMALL,19.0,16.5,MEDIUM,GOOD,MEDIUM_GOOD,2,2,2,1,2,9,LARGE,0,NaN,0.0,0.0
4577,446696,2,37,43,35,4,커머스,7,1369916,51.514286,41.0,ADS_CODE_DAILY_UPDATE,18,14,2025-08-24 04:10:55,2025-08-25 06:25:34,멜피움 스팀청소기 물걸레 고온 살균 청소 179815,2025-08-24 04:08:25,0,2,17.5,0.8,4,0.3,140,SMALL,21.5,18.5,LARGE,GOOD,LARGE_GOOD,2,2,2,1,2,9,LARGE,0,NaN,0.0,0.0
4578,446697,2,136,174,131,4,커머스,7,1369916,73.244275,52.0,ADS_CODE_DAILY_UPDATE,18,14,2025-08-24 04:10:57,2025-08-25 04:09:32,스팀다리미 핸디 여행용 미니 핸디형스팀다리미 179818,2025-08-24 04:08:26,0,1,131.0,0.8,4,0.3,524,MEDIUM,174.0,136.0,MEGA,HIGH,MEGA_HIGH,2,3,3,0,2,10,LARGE,0,NaN,0.0,0.0
4579,446698,3,289,370,267,4,커머스,7,1369916,69.329588,56.0,ADS_CODE_DAILY_UPDATE,18,14,2025-08-24 04:16:14,2025-08-25 11:15:24,한경희 핸디 소형청소기 무선 미니 청소기 179820,2025-08-24 04:12:33,0,2,133.5,0.7,4,0.3,1068,LARGE,185.0,144.5,MEGA,HIGH,MEGA_HIGH,2,3,3,1,1,10,LARGE,0,NaN,0.0,0.0
4580,446699,2,140,176,127,4,커머스,7,1369916,73.913386,54.0,ADS_CODE_DAILY_UPDATE,18,14,2025-08-24 04:20:41,2025-08-25 08:53:43,25W C타입충전기 PD 초고속 여행용 핸드폰 고속충전기 어댑터 179823,2025-08-24 04:17:24,0,2,63.5,0.7,4,0.3,508,MEDIUM,88.0,70.0,MEGA,HIGH,MEGA_HIGH,2,3,3,1,1,10,LARGE,0,NaN,0.0,0.0
4581,446700,2,154,193,144,4,커머스,7,1369916,62.736111,49.0,ADS_CODE_DAILY_UPDATE,18,14,2025-08-24 04:25:58,2025-08-25 09:24:31,갤럭시 아이폰 무선충전기 갤럭시워치 애플워치 멀티 여행용 무선충전기 179827,2025-08-24 04:22:35,0,2,72.0,0.7,4,0.3,576,MEDIUM,96.5,77.0,MEGA,HIGH,MEGA_HIGH,2,3,3,1,1,10,LARGE,0,NaN,0.0,0.0


In [139]:
print(104/191, 39/191)

0.5445026178010471 0.20418848167539266


In [140]:
ads_pool.loc[(ads_pool['ads_category'] == 4) & (ads_pool.ads_rejoin_type == 'NONE') & (ads_pool['mda_idx_arr'].isna()) & (ads_pool['media_count'] >= 2) #,
             ] # .value_counts('domain') # ['contract_price', 'media_price'].median()

,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,ads_order,ctit_mean,ctit_median,ads_rejoin_type,contract_price,media_price,first_click,last_click,ads_name,ads_sdate,expire,days_active,daily_avg_conversions,cvr,margin,roi,total_net_return,ads_level_detailed,daily_clicks,daily_users,scale_level,performance_level,ads_level,media_score,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster,mda_idx_arr,M,A
1004,431912,2,5,6,1,4,커머스,7,1369551,70.000000,70.0,NONE,17,8,2025-07-28 03:41:15,2025-08-12 06:11:14,보스턴백 (리뷰 전체보기 클릭 후 URL 제출),2025-07-18 19:12:12,0,16,0.062500,0.2,9,1.1,9,MICRO,0.375000,0.312500,MEDIUM,LOW,MEDIUM_LOW,2,0,0,3,0,5,MEDIUM,0,NaN,0.0,1.0
1030,434507,2,20,29,13,4,식당/카페,7,1369550,83.000000,91.0,NONE,19,8,2025-07-31 10:34:46,2025-08-18 23:12:34,스시타다시 (블로그 리뷰 클릭 후 플레이스 저장),2025-07-21 11:07:30,0,19,0.684211,0.4,11,1.4,143,SMALL,1.526316,1.052632,MEDIUM,LOW,MEDIUM_LOW,2,0,0,3,0,5,MEDIUM,0,NaN,0.0,1.0
1041,434578,2,4,6,0,4,커머스,7,1369610,NaN,NaN,NONE,16,8,2025-08-04 14:28:01,2025-08-04 15:09:07,인절미 (상품 태그 맞추기),2025-07-21 14:57:23,0,1,0.000000,0.0,8,1.0,0,MICRO,6.000000,4.000000,MEDIUM,LOW,MEDIUM_LOW,2,0,1,0,0,3,SMALL,0,NaN,0.0,1.0
1051,434735,2,6,6,3,4,커머스,7,1369610,64.333333,64.0,NONE,17,8,2025-07-28 02:14:05,2025-08-20 03:16:52,한우국거리 (리뷰 전체보기 클릭 후 URL 제출),2025-07-22 00:07:53,0,24,0.125000,0.5,9,1.1,27,MICRO,0.250000,0.250000,MEDIUM,LOW,MEDIUM_LOW,2,0,0,3,0,5,MEDIUM,0,NaN,0.0,1.0
1052,434737,2,3,3,1,4,커머스,7,1369610,114.000000,114.0,NONE,17,8,2025-08-17 19:38:31,2025-08-22 07:10:13,한우선물세트 (리뷰 전체보기 클릭 후 URL 제출),2025-07-22 00:07:55,0,5,0.200000,0.3,9,1.1,9,MICRO,0.600000,0.600000,MEDIUM,LOW,MEDIUM_LOW,2,0,0,2,0,4,SMALL,0,NaN,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4647,446810,3,409,533,60,4,식당/카페,7,1369916,81.083333,40.0,NONE,20,17,2025-08-24 16:04:59,2025-08-24 23:57:27,[플레이스 퀴즈] 맘보식당 해운대송정본점 (18780),2025-08-24 15:57:56,0,1,60.000000,0.1,3,0.2,180,LARGE,533.000000,409.000000,MEGA,HIGH,MEGA_HIGH,2,3,4,0,0,9,LARGE,0,NaN,0.0,0.0
4648,446811,2,39,50,27,4,식당/카페,7,1369916,54.814815,53.0,NONE,20,17,2025-08-24 16:00:26,2025-08-24 16:06:14,[플레이스 퀴즈] 왐왐커피 (18784),2025-08-24 15:57:56,0,1,27.000000,0.5,3,0.2,81,MEDIUM,50.000000,39.000000,LARGE,GOOD,LARGE_GOOD,2,2,2,0,0,6,MEDIUM,0,NaN,0.0,0.0
4649,446812,2,28,31,20,4,식당/카페,7,1369916,45.300000,36.5,NONE,20,17,2025-08-24 16:00:44,2025-08-24 16:04:42,[플레이스 퀴즈] 우리동네 소고기무한리필 부천역점 (18791),2025-08-24 15:57:57,0,1,20.000000,0.6,3,0.2,60,MEDIUM,31.000000,28.000000,LARGE,GOOD,LARGE_GOOD,2,2,2,0,1,7,MEDIUM,0,NaN,0.0,0.0
4650,446813,2,2,2,2,4,운동/스포츠,7,1369916,28.500000,28.5,NONE,16,11,2025-08-25 00:18:40,2025-08-25 00:33:31,골프피규어 맞추기 8.24,2025-08-24 17:12:13,0,1,2.000000,1.0,5,0.5,10,MICRO,2.000000,2.000000,MEDIUM,HIGH,MEDIUM_HIGH,2,1,0,0,4,7,MEDIUM,0,NaN,0.0,0.0


In [162]:
ads_list[ads_list.ads_idx.isin([431912, 434735, 434737])]

,ads_idx,ads_code,aff_idx,adv_idx,ads_type,ads_category,ads_name,ads_icon_img,ads_summary,ads_save_way,ads_day_cap,ads_sdate,ads_edate,ads_age_min,ads_age_max,ads_os_type,ads_contract_price,ads_reward_price,ads_order,ads_rejoin_type,regdate,domain
1132,431912,HMGtt02Ti0,61,82,3,4,보스턴백 (리뷰 전체보기 클릭 후 URL 제출),https://img.greenp.kr/cpi/17528151112501.jpg,***상품 접속 후 상품 리뷰보기 클릭 &gt; 리뷰 전체보기 클릭 (리뷰 전체보기...,리뷰 전체보기 클릭 후 URL 제출,False,2025-07-18 19:12:12,9999-12-31 23:59:59,0,100,7,17,8,1369551,NONE,2025-07-18 19:12:12 UTC,커머스
1352,434735,2ctQ2tqcmm,61,82,3,4,한우국거리 (리뷰 전체보기 클릭 후 URL 제출),https://img.greenp.kr/cpi/17528151112501.jpg,***상품 접속 후 상품 리뷰보기 클릭 &gt; 리뷰 전체보기 클릭 (리뷰 전체보기...,리뷰 전체보기 클릭 후 URL 제출,False,2025-07-22 00:07:53,9999-12-31 23:59:59,0,100,7,17,8,1369610,NONE,2025-07-22 00:07:53 UTC,커머스
1353,434737,vuAyGNuGdV,61,82,3,4,한우선물세트 (리뷰 전체보기 클릭 후 URL 제출),https://img.greenp.kr/cpi/17530630576939.jpg,***상품 접속 후 상품 리뷰보기 클릭 &gt; 리뷰 전체보기 클릭 (리뷰 전체보기...,리뷰 전체보기 클릭 후 URL 제출,False,2025-07-22 00:07:55,9999-12-31 23:59:59,0,100,7,17,8,1369610,NONE,2025-07-22 00:07:55 UTC,커머스


In [128]:
analyze_ads_performance(179092, click)

,ads_idx,mda_idx,total_clicks,total_conversions,contract_price,media_price,domain,ads_category,cvr,profit_per_conversion,total_profit,first_click,last_click,days_active_calc,daily_clicks,daily_conversions,daily_profit,배분그룹
0,179092,371,79,32,13,9,뷰티,4,0.4051,4,128,2025-07-26 19:48:19,2025-08-21 23:23:46,27,2.925926,1.185185,4.740741,분류불가


In [146]:
ads_list.ads_idx.agg({'max', 'min'})

min      6508
max    446909
Name: ads_idx, dtype: int64

### 금융/구독

In [188]:
ads_pool[ads_pool['ads_category'] == 7]

,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,ads_order,ctit_mean,ctit_median,ads_rejoin_type,contract_price,media_price,first_click,last_click,ads_name,ads_sdate,expire,days_active,daily_avg_conversions,cvr,margin,roi,total_net_return,ads_level_detailed,daily_clicks,daily_users,scale_level,performance_level,ads_level,media_score,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster,mda_idx_arr,M,A
1,6985,2,30,46,0,7,금융,7,1201938,NaN,NaN,NONE,2700,2300,2025-07-26 11:02:18,2025-08-24 05:39:13,DB손해보험 다이렉트,2020-06-30 15:00:00,0,29,0.000000,0.0,400,0.2,0,MEDIUM,1.586207,1.034483,MEDIUM,LOW,MEDIUM_LOW,2,0,0,3,0,5,MEDIUM,0,NaN,0.0,0.0
29,13988,9,32,55,2,7,금융,7,1554607,41.500000,41.5,NONE,6000,4500,2025-07-26 23:17:18,2025-08-24 13:56:08,주식투자 정보서비스,2021-11-16 13:00:00,0,29,0.068966,0.0,1500,0.3,3000,LARGE,1.896552,1.103448,LARGE,LOW,LARGE_LOW,3,0,0,3,0,6,MEDIUM,0,NaN,0.0,1.0
45,16730,4,191,314,6,7,금융,7,1360091,1509.333333,544.0,NONE,5000,3750,2025-07-26 10:19:39,2025-08-24 15:14:59,KB차차차,2022-04-25 19:00:00,0,30,0.200000,0.0,1250,0.3,7500,LARGE,10.466667,6.366667,LARGE,LOW,LARGE_LOW,3,0,1,3,0,7,MEDIUM,0,NaN,0.0,1.0
151,27572,8,20,25,1,7,금융,7,2289200,165.000000,165.0,NONE,600,315,2025-07-26 14:30:52,2025-08-24 16:52:09,신한 SOL페이 퀴즈팡팡 퀴즈 풀면,2023-12-21 15:00:00,0,30,0.033333,0.0,285,0.9,285,LARGE,0.833333,0.666667,LARGE,LOW,LARGE_LOW,3,0,0,3,0,6,MEDIUM,0,NaN,0.0,1.0
167,27889,3,49,71,0,7,금융,7,1673919,NaN,NaN,NONE,4000,3000,2025-07-26 09:21:41,2025-08-18 22:31:22,KB다이렉트 자동차보험,2024-01-10 16:00:00,0,24,0.000000,0.0,1000,0.3,0,MEDIUM,2.958333,2.041667,MEDIUM,LOW,MEDIUM_LOW,2,0,1,3,0,6,MEDIUM,0,NaN,0.0,1.0
172,28014,8,81,141,0,7,금융,2,1667243,NaN,NaN,NONE,3000,2250,2025-07-26 02:44:34,2025-08-25 06:47:35,펀드정보,2024-01-17 15:00:00,0,31,0.000000,0.0,750,0.3,0,LARGE,4.548387,2.612903,LARGE,LOW,LARGE_LOW,3,0,1,4,0,8,LARGE,0,NaN,0.0,1.0
193,31140,6,23,30,2,7,금융,7,2529500,151.500000,151.5,NONE,1000,525,2025-07-26 17:47:42,2025-08-25 09:07:03,신한라이프 마케팅동의 이벤트,2024-04-05 10:00:00,0,30,0.066667,0.1,475,0.9,950,LARGE,1.000000,0.766667,LARGE,LOW,LARGE_LOW,3,0,0,3,0,6,MEDIUM,0,NaN,0.0,1.0
279,72966,8,69,85,0,7,금융,7,2684000,NaN,NaN,NONE,30000,18000,2025-07-26 09:35:31,2025-08-25 06:56:12,KB다이렉트 간편실손보험,2024-07-18 17:00:00,0,30,0.000000,0.0,12000,0.7,0,LARGE,2.833333,2.300000,LARGE,LOW,LARGE_LOW,3,0,1,3,0,7,MEDIUM,0,NaN,0.0,1.0
280,72967,9,59,89,1,7,금융,7,2591300,303.000000,303.0,NONE,30000,12000,2025-07-26 19:27:19,2025-08-25 00:03:56,KB다이렉트 암보험,2024-07-18 17:00:00,0,30,0.033333,0.0,18000,1.5,18000,LARGE,2.966667,1.966667,LARGE,LOW,LARGE_LOW,3,0,1,3,0,7,MEDIUM,0,NaN,0.0,1.0
281,72968,4,22,33,0,7,금융,7,2642900,NaN,NaN,NONE,30000,10000,2025-07-27 01:26:06,2025-08-18 18:34:36,KB다이렉트 운전자보험,2024-07-18 17:00:00,0,23,0.000000,0.0,20000,2.0,0,LARGE,1.434783,0.956522,LARGE,LOW,LARGE_LOW,3,0,0,3,0,6,MEDIUM,0,NaN,0.0,1.0


In [193]:
ads_list[ads_list['ads_category'] == 7] # .value_counts('ads_type')

,ads_idx,ads_code,aff_idx,adv_idx,ads_type,ads_category,ads_name,ads_icon_img,ads_summary,ads_save_way,ads_day_cap,ads_sdate,ads_edate,ads_age_min,ads_age_max,ads_os_type,ads_contract_price,ads_reward_price,ads_order,ads_rejoin_type,regdate,domain
1,6985,SijDrtH4GV,13,335,3,7,DB손해보험 다이렉트,https://seimg.pincrux.com/icon/1575337232.jpg,[참여방법]\n- 빠르고 간편한 DB손해보험 다이렉트에서\n- 내 차 보험료를 지금...,내차 보험료 조회,False,2020-06-30 15:00:00,9999-06-06 00:00:00,0,100,7,2700,2300,1201938,NONE,2020-06-30 15:22:59 UTC,금융
32,13988,nTVtBN47OC,1,404,3,7,주식투자 정보서비스,https://i.imgur.com/N6KVHsQ.jpg,[참여 방법]\n1. 광고 참여 버튼 터치하여 이벤트 페이지로 이동\n2. 이벤트 ...,이벤트 참여,False,2021-11-16 13:00:00,2030-01-01 00:00:00,0,100,7,6000,4500,1554607,NONE,2021-11-16 13:53:11 UTC,금융
48,16730,cwnAuYWkOH,13,335,3,7,KB차차차,https://seimg.pincrux.com/icon/1650850739.png,[참여방법]\n1. 앱 오픈 후 회원가입 하기\n2. 앱 우측 하단 차테크 터치\n...,최초 오픈 후 금융자산 연결,False,2022-04-25 19:00:00,2027-01-01 00:00:00,0,100,7,5000,3750,1360091,NONE,2022-04-25 19:12:07 UTC,금융
158,27572,955oLvqtif,23,220,3,7,신한 SOL페이 퀴즈팡팡 퀴즈 풀면,https://i.imgur.com/D6tOnD7.png,퀴즈팡팡 OX퀴즈 (최근 1년 이상 신한 SOL페이(구 신한pLay)에 방문한 적이...,이벤트 참여 완료,False,2023-12-21 15:00:00,2029-11-28 00:00:00,0,100,7,600,450,2289200,NONE,2023-12-21 15:38:56 UTC,금융
174,27889,lw0bEsbt7j,1,404,3,7,KB다이렉트 자동차보험,https://i.imgur.com/KfeTUS5.png,[자동차보험 견적조회] KB다이렉트 자동차보험\n\n[이벤트 참여대상]\n- 본인 ...,견적조회,False,2024-01-10 16:00:00,2029-11-29 00:00:00,0,100,7,4000,3000,1673919,NONE,2024-01-10 16:31:08 UTC,금융
179,28014,Vm1LhWmCnj,75,404,3,7,펀드정보,https://i.imgur.com/GfnmRAH.png,[참여방법]\n 1. 휴대폰 본인인증\n 2. T펀드정보 서비스 가입\n\n[유의사...,서비스 가입,False,2024-01-17 15:00:00,2029-11-29 00:00:00,0,100,2,3000,2250,1667243,NONE,2024-01-17 15:27:53 UTC,금융
203,31140,zsRVnwjwJE,23,220,3,7,신한라이프 마케팅동의 이벤트,https://i.imgur.com/gkBUUoF.png,마케팅동의 하고 혜택 받으세요.\n\n[참여 방법]\n1. 광고 참여 버튼 터치하여...,참여 하기,False,2024-04-05 10:00:00,2029-11-29 00:00:00,0,100,7,1000,750,2529500,NONE,2024-04-05 10:26:39 UTC,금융
289,72966,rabqtGN8ZC,1,753,3,7,KB다이렉트 간편실손보험,https://i.imgur.com/xqxHHyL.png,"KB다이렉트 간편실손보험 가입 시 리워드 지급!\n*중요* 월납보험료 15,000원...",KB다이렉트 간편실손보험 가입,False,2024-07-18 17:00:00,2030-01-01 00:00:00,0,100,7,30000,18000,2684000,NONE,2024-07-18 16:19:14 UTC,금융
290,72967,EeHd8iDRVk,1,753,3,7,KB다이렉트 암보험,https://i.imgur.com/ITxz6XI.png,KB다이렉트 암보험 가입 시 리워드 지급!\n*중요* 월납보험료 1만원 이상일 경우...,KB다이렉트 암보험 가입,False,2024-07-18 17:00:00,2030-01-01 00:00:00,0,100,7,30000,12000,2591300,NONE,2024-07-18 16:19:33 UTC,금융
291,72968,j6rAtzplRm,1,753,3,7,KB다이렉트 운전자보험,https://i.imgur.com/sVE9wYK.png,"KB다이렉트 운전자보험 가입 시 리워드 지급!\n*중요* 월납보험료 8,500원 이...",KB다이렉트 운전자보험 가입,False,2024-07-18 17:00:00,2030-01-01 00:00:00,0,100,7,30000,10000,2642900,NONE,2024-07-18 16:19:54 UTC,금융


In [189]:
ads_pool[ads_pool['ads_category'] == 3]

,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,ads_order,ctit_mean,ctit_median,ads_rejoin_type,contract_price,media_price,first_click,last_click,ads_name,ads_sdate,expire,days_active,daily_avg_conversions,cvr,margin,roi,total_net_return,ads_level_detailed,daily_clicks,daily_users,scale_level,performance_level,ads_level,media_score,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster,mda_idx_arr,M,A
6,9982,36,22510,23187,12981,3,미디어/컨텐츠,7,2729900,114.674062,118.0,NONE,230,160,2025-07-26 03:09:06,2025-08-25 11:23:42,강원일보 네이버 뉴스,2021-02-05 14:00:00,0,31,418.741935,0.6,70,0.4,908670,MEGA,747.967742,726.129032,MEGA,HIGH,MEGA_HIGH,4,4,4,4,1,17,MEGA,0,NaN,0.0,1.0
60,19488,97,12099,12337,6372,3,미디어/컨텐츠,2,2729100,90.032486,103.0,NONE,230,160,2025-07-26 06:09:04,2025-08-25 06:39:19,광주방송 네이버 뉴스,2022-09-23 12:00:00,0,31,205.548387,0.5,70,0.4,446040,MEGA,397.967742,390.290323,MEGA,HIGH,MEGA_HIGH,4,4,3,4,0,15,MEGA,0,NaN,0.0,1.0
110,21448,2,532,731,300,3,금융,7,2487000,40.550000,30.0,NONE,200,160,2025-08-11 09:15:46,2025-08-18 17:12:15,신한카드 유튜브,2023-04-20 15:00:00,0,8,37.500000,0.4,40,0.2,12000,LARGE,91.375000,66.500000,MEGA,GOOD,MEGA_GOOD,2,2,3,2,0,9,LARGE,0,NaN,0.0,1.0
185,28947,8,179,302,90,3,채용,7,2745900,57.200000,43.5,NONE,230,180,2025-07-26 23:51:34,2025-08-25 08:53:19,쿠팡 로지스틱스 채용,2024-02-27 10:00:00,0,30,3.000000,0.3,50,0.3,4500,LARGE,10.066667,5.966667,LARGE,LOW,LARGE_LOW,3,1,1,3,0,8,LARGE,0,NaN,0.0,1.0
217,34045,10,5749,16083,727,3,미디어/컨텐츠,7,2729400,8071.803301,76.0,NONE,230,160,2025-07-26 06:16:06,2025-08-25 11:21:18,이마트 유튜브,2024-04-19 15:00:00,0,31,23.451613,0.0,70,0.4,50890,MEGA,518.806452,185.451613,MEGA,GOOD,MEGA_GOOD,3,2,4,4,0,13,MEGA,0,NaN,0.0,1.0
256,56225,40,1731,1995,1343,3,생활,7,2729000,149.570365,178.0,NONE,230,180,2025-07-26 10:17:59,2025-08-25 11:19:08,LG전자 인스타그램,2024-06-25 11:00:00,0,31,43.322581,0.7,50,0.3,67150,MEGA,64.354839,55.838710,MEGA,HIGH,MEGA_HIGH,4,3,2,4,1,14,MEGA,0,NaN,0.0,1.0
448,155357,4,2319,3501,1645,3,뷰티,2,2453500,467.745289,29.0,NONE,230,180,2025-07-29 09:09:28,2025-08-07 12:20:46,스나이델뷰티 인스타그램,2025-02-13 18:00:00,0,10,164.500000,0.5,50,0.3,82250,LARGE,350.100000,231.900000,MEGA,HIGH,MEGA_HIGH,3,3,3,2,0,11,LARGE,0,NaN,0.0,1.0
828,393760,4,34,51,3,3,식음료,1,2592200,87.666667,30.0,NONE,230,180,2025-08-20 15:43:05,2025-08-25 11:09:50,풀무원 PM1 스튜디오 유튜브,2025-06-20 11:00:00,0,5,0.600000,0.1,50,0.3,150,MEDIUM,10.200000,6.800000,LARGE,LOW,LARGE_LOW,3,0,1,2,0,6,MEDIUM,0,NaN,0.0,1.0
829,393761,9,563,849,176,3,식음료,2,2592300,77.698864,43.0,NONE,230,180,2025-08-20 14:07:03,2025-08-25 11:22:16,풀무원 PM1 스튜디오 유튜브,2025-06-20 11:00:00,0,5,35.200000,0.2,50,0.3,8800,LARGE,169.800000,112.600000,MEGA,GOOD,MEGA_GOOD,3,2,3,2,0,10,LARGE,0,NaN,0.0,1.0
3489,442071,37,8986,9728,8245,3,뷰티,7,2731200,140.740813,47.0,NONE,230,180,2025-08-08 18:26:15,2025-08-11 10:45:44,알럭스 인스타그램,2025-08-08 18:00:00,0,3,2748.333333,0.8,50,0.3,412250,MEGA,3242.666667,2995.333333,MEGA,HIGH,MEGA_HIGH,4,4,4,1,2,15,MEGA,0,NaN,0.0,1.0


In [192]:
ads_list[ads_list['ads_category'] == 3] # .value_counts('ads_type')

,ads_idx,ads_code,aff_idx,adv_idx,ads_type,ads_category,ads_name,ads_icon_img,ads_summary,ads_save_way,ads_day_cap,ads_sdate,ads_edate,ads_age_min,ads_age_max,ads_os_type,ads_contract_price,ads_reward_price,ads_order,ads_rejoin_type,regdate,domain
6,9982,ChpB9DGeNo,8,50,11,3,강원일보 네이버 뉴스,https://static.adbrix.igaworks.com/adpopcorn/2...,[참여방법]\n1.\'참여하기\' 버튼 터치하고 이벤트페이지로 이동\n2. 이벤트 ...,네이버 뉴스 구독,False,2021-02-05 14:00:00,2030-01-01 00:00:00,0,100,7,230,160,2729900,NONE,2021-02-05 14:56:37 UTC,미디어/컨텐츠
26,11124,RoPyXNtRjb,1,866,3,3,[친한친구] 베지밀 공식 인스타그램,https://i.imgur.com/cL5BfYO.jpg,[참여방법]\n\'팔로우\' 버튼 클릭 → \'팔로잉v\' 버튼 클릭→\'친한 친구...,팔로우 후 친한 친구 리스트 추가,False,2021-05-10 00:00:00,2025-08-12 00:00:00,0,100,7,280,120,2759800,NONE,2021-05-10 11:28:45 UTC,식음료
27,11442,RoPyXNtRjb,1,866,3,3,[친한친구] 베지밀 공식 인스타그램,https://i.imgur.com/cL5BfYO.jpg,[참여방법]\n\'팔로우\' 버튼 클릭 → \'팔로잉v\' 버튼 클릭→\'친한 친구...,팔로우 후 친한 친구 리스트 추가,False,2021-06-18 00:00:00,2025-08-19 00:00:00,0,100,7,280,120,2760000,NONE,2021-06-08 14:04:47 UTC,식음료
59,18504,GPOUSbWnaX,30,866,5,3,정식품 페이스북,https://buzzvil.akamaized.net/adfit.image/uplo...,페이스북 페이지를 좋아요 하세요.\n\n[참여방법]\n- \'좋아요\' 를 완료하시...,좋아요,False,2022-07-19 15:00:00,2025-08-20 00:00:00,0,100,7,257,180,2760500,NONE,2022-07-19 15:30:29 UTC,식음료
64,19488,tBwR3QXYKy,8,50,11,3,광주방송 네이버 뉴스,https://i.imgur.com/73IZoOo.jpg,광주방송 네이버 뉴스 \'구독\'하면 리워드 지급\n* 7일 이내 \'구독\' 취소...,네이버뉴스 구독,True,2022-09-23 12:00:00,2030-01-01 00:00:00,0,100,2,230,160,2729100,NONE,2022-09-23 11:55:06 UTC,미디어/컨텐츠
116,21448,lMziFxy0nV,71,910,10,3,신한카드 유튜브,https://cdnet.nasmob.com/nstation/image/202303...,신한카드 유튜브 계정을 구독하세요.\n유튜브 채널 구독+좋아요를 누르시고 참여완료 ...,구독 + 좋아요,False,2023-04-20 15:00:00,2031-01-03 00:00:00,0,100,7,230,170,2487000,NONE,2023-04-20 15:27:54 UTC,금융
195,28947,fLFW3uBd3f,8,50,3,3,쿠팡 로지스틱스 채용,https://i.imgur.com/XbUGVm2.jpg,쿠팡로지스틱스 채용 카카오톡 채널 \'추가\'하고 혜택받자!\n20세 ~ 59세 신...,카카오톡 채널추가,False,2024-02-27 10:00:00,2027-05-01 00:00:00,20,54,7,230,180,2745900,NONE,2024-02-27 10:14:31 UTC,채용
227,34045,IhvxEGKBOh,8,50,10,3,이마트 유튜브,https://static.adbrix.igaworks.com/adpopcorn/2...,이마트 유튜브 채널 \'구독 및 좋아요\'하면 리워드 지급\n(20~59세만 참여 ...,구독 및 좋아요,False,2024-04-19 15:00:00,2029-11-01 00:00:00,20,59,7,230,160,2729400,NONE,2024-04-19 15:31:37 UTC,미디어/컨텐츠
266,56225,xFwInhitco,8,50,7,3,LG전자 인스타그램,https://static.adbrix.igaworks.com/adpopcorn/2...,인스타그램 \'팔로우\'하면 리워드 지급\n(19~44세 유저만 참여 가능합니다.)...,팔로우,False,2024-06-25 11:00:00,2029-11-29 00:00:00,0,100,7,230,180,2729000,NONE,2024-06-25 11:45:34 UTC,생활
480,155357,EW0ULr5bno,8,50,7,3,스나이델뷰티 인스타그램,https://static.adbrix.igaworks.com/adpopcorn/2...,인스타그램 \'팔로우\'하면 리워드 지급\n* 25~44세 여성 유저만 참여 가능합...,팔로우,False,2025-02-13 18:00:00,2031-01-04 00:00:00,0,100,2,230,180,2453500,NONE,2025-02-13 18:51:00 UTC,뷰티


### Make examples

리스트에 들어가야 할 것들:

ads_idx, ads_type, ads_category, ads_name, ads_os_type, ads_contract_price, ads_reward_price, ads_rejoin_type, domain

In [213]:
# ads_idx, ads_type, ads_category, ads_name, ads_os_type, ads_contract_price, ads_reward_price, ads_rejoin_type, domain
new_ads = pd.DataFrame({'ads_idx':[500000, 500001, 500002, 500003, 500004, 500005, 500006, 500007], 'ads_type':[12, 3, 3, 11, 7, 3, 3, 3], 'ads_category':[11, 4, 4, 3, 3, 7, 5, 5], 'ads_name':['청정원 선물세트', '옹짬뽕 포항양덕본점', '추석선물세트 (리뷰 전체보기 클릭 후 URL 제출)', "네이버 뉴스 제주뉴스 구독", "폴라베어 인스타그램 구독", "스마트페이 계좌개설",
                                                                                                                                                                     "토리 키우기 (Lv.3까지)", "드래곤사가 신규 영웅 뽑기"],
                        'ads_os_type':[7, 7, 7, 7, 1, 7, 1, 2], 'ads_contract_price':[6181, 18, 17, 230, 230, 6400, 1600, 6700], 'ads_media_price':[4000, 14, 8, 160, 160, 4800, 1200, 5000 ], 'ads_reward_price':[4000, 12, 8, 180, 180, 4800, 1200, 5000], 'ads_rejoin_type':['REJOINABLE', 'ADS_CODE_DAILY_UPDATE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE'] ,
                        'domain': ['식음료', '식당/카페', '식음료', '미디어/컨텐츠', '커머스', '금융', '게임', '게임']})
new_ads

,ads_idx,ads_type,ads_category,ads_name,ads_os_type,ads_contract_price,ads_media_price,ads_reward_price,ads_rejoin_type,domain
0,500000,12,11,청정원 선물세트,7,6181,4000,4000,REJOINABLE,식음료
1,500001,3,4,옹짬뽕 포항양덕본점,7,18,14,12,ADS_CODE_DAILY_UPDATE,식당/카페
2,500002,3,4,추석선물세트 (리뷰 전체보기 클릭 후 URL 제출),7,17,8,8,NONE,식음료
3,500003,11,3,네이버 뉴스 제주뉴스 구독,7,230,160,180,NONE,미디어/컨텐츠
4,500004,7,3,폴라베어 인스타그램 구독,1,230,160,180,NONE,커머스
5,500005,3,7,스마트페이 계좌개설,7,6400,4800,4800,NONE,금융
6,500006,3,5,토리 키우기 (Lv.3까지),1,1600,1200,1200,NONE,게임
7,500007,3,5,드래곤사가 신규 영웅 뽑기,2,6700,5000,5000,NONE,게임


In [214]:
new_ads.to_csv('신규가상광고.csv', index = False)

In [145]:
ads_list # ads_idx, ads_type, ads_category, ads_name, ads_os_type, ads_contract_price, ads_reward_price, ads_rejoin_type, domain

,ads_idx,ads_code,aff_idx,adv_idx,ads_type,ads_category,ads_name,ads_icon_img,ads_summary,ads_save_way,ads_day_cap,ads_sdate,ads_edate,ads_age_min,ads_age_max,ads_os_type,ads_contract_price,ads_reward_price,ads_order,ads_rejoin_type,regdate,domain
0,6508,3u4lOKJAek,13,335,3,10,스마트피싱보호,https://seimg.pincrux.com/icon/1551770735.png,"[상품소개]\n알면 피하고, 모르면 당하는 보이스피싱\n나와 소중한 가족을 안전하게...",서비스 가입,False,2020-04-23 17:00:00,9999-12-31 00:00:00,0,100,7,1400,1100,2381500,NONE,2020-04-23 17:53:01 UTC,금융
1,6985,SijDrtH4GV,13,335,3,7,DB손해보험 다이렉트,https://seimg.pincrux.com/icon/1575337232.jpg,[참여방법]\n- 빠르고 간편한 DB손해보험 다이렉트에서\n- 내 차 보험료를 지금...,내차 보험료 조회,False,2020-06-30 15:00:00,9999-06-06 00:00:00,0,100,7,2700,2300,1201938,NONE,2020-06-30 15:22:59 UTC,금융
2,8327,k1RZftVHoW,13,335,3,10,파일썬,https://static.i-screen.kr/img/ad/icon/aff_13_...,[참여 방법]\n1. 광고 참여 버튼 터치하여 이벤트 페이지로 이동\n2. 이벤트 ...,첫 결제,False,2020-11-10 11:21:00,9999-12-31 23:59:00,0,100,7,3200,2420,1673458,NONE,2020-11-10 11:21:43 UTC,미디어/컨텐츠
3,9264,EGZcaVOtlc,23,220,3,10,하나 가득담은 운전자보험가입,https://i.imgur.com/kaWiYyT.png,하나손해보험 다이렉트 운전자보험 가입시\n\n01.참여방법\n1) 계산하기 페이지를...,보험 가입 완료,False,2020-11-30 00:00:00,9999-12-31 23:59:59,0,100,7,11000,8250,2372800,NONE,2020-11-30 15:16:45 UTC,금융
4,9716,NwZZq45Kgb,6,10,3,8,베이비러브 참여 신청,https://nextapps-nas.aws.appang.kr/icon/SDl2uC...,[참여방법]\n- 페이지 이동 > 베이버러브 참여신청 완료\n- 25-55세 SKT...,참여신청 (25세 ~ 55세),False,2021-01-11 16:00:00,2029-11-29 00:00:00,25,55,2,600,480,2699900,NONE,2021-01-11 16:11:27 UTC,생활
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9512,446899,U80Aed1tRq,86,1655,3,4,원스토몰 180212,https://cashplan-r2.uk/quiz-mission.png,[참여방법]\n1. 본문에서 퀴즈와 상품/장소 확인\n1-1. 퀴즈: 4번째 태그 ...,네이버스마트스토어 퀴즈,False,2025-08-25 09:37:27,9999-12-31 23:59:59,0,100,7,18,12,1369916,ADS_CODE_DAILY_UPDATE,2025-08-25 09:37:27 UTC,커머스
9513,446900,dfX4EjOsTE,86,1655,3,4,서빙카트 식당 업소용 주방 서빙카 다용도 서빙카트 소형 2단 기본고무바퀴 180213,https://cashplan-r2.uk/quiz-mission.png,[참여방법]\n1. 본문에서 퀴즈와 상품/장소 확인\n1-1. 퀴즈: 7번째 태그 ...,네이버스마트스토어 퀴즈,False,2025-08-25 09:42:08,9999-12-31 23:59:59,0,100,7,18,12,1369915,ADS_CODE_DAILY_UPDATE,2025-08-25 09:42:08 UTC,식당/카페
9514,446902,rc7EZrAoQw,57,84,3,8,[네이버카페가입인사] 대장TV,https://lh3.googleusercontent.com/Zt1Ac3OOTTTa...,<< 적립방법 >>\n\n1. 최초 참여시 네이버 로그인 진행\n\n2. 카페 가입...,참여,False,2025-08-25 10:00:00,2030-12-27 00:00:00,0,100,7,200,150,2735900,NONE,2025-08-25 10:06:15 UTC,미디어/컨텐츠
9515,446908,HRMcs0VIjL,13,335,2,0,NOL,https://seimg.pincrux.com/icon/1756085578.5054...,[참여방법]\n- 앱에 접속하시면 리워드가 지급됩니다.\n\n[주의사항]\n- 리워...,최초 오픈,False,2025-08-25 10:37:11,9999-12-31 23:59:59,0,100,2,230,180,1682806,NONE,2025-08-25 10:37:10 UTC,금융


In [ ]:
ads_pool.loc[(ads_pool['ads_category'] == 4) & (ads_pool.ads_rejoin_type == 'NONE')]

In [103]:
ads_pool[ads_pool['ads_os_type'] == 1].shape

(90, 42)